In [ ]:
import tensorflow as tf

import nobrainer
from nobrainer import dataset, volume

from sklearn import preprocessing
min_max_scaler = preprocessing.MinMaxScaler()

dir_path = os.path.abspath("csv/faced_defaced/train_test_fold_1/csv/")
csv_path = os.path.join(dir_path, "training.csv")
# print(dir_path, csv_path)


labels = pd.read_csv(csv_path)["Y"].values
paths = pd.read_csv(csv_path)["X"].values


# print(labels)

n_classes=2
volume_shape = (256, 256, 256)
block_shape = (128, 128, 128)


training_paths = zip(paths, labels)

print(training_paths)

In [1]:
import nobrainer

In [ ]:
/work/01329/poldrack/data/mriqc-net/data/masks/

In [ ]:
import os
import glob
import subprocess
import time

orig_root_dir = '/work/01329/poldrack/data/mriqc-net/data'
orig_face_defaced = os.path.join(orig_root_dir, 'masks')
orig_data_deface = os.path.join(orig_root_dir, 'defaced')

orig_reface_dir = '/work/06850/sbansal6/maverick2/mriqc-shared/refaced'
save_orig_faced_root_dir = os.path.join(orig_reface_dir, 'orig_faced')

simg_path = os.path.join(orig_reface_dir,'afni-latest.simg')
print(simg_path)
exclude = ['ds002033_anat', 'ds002149_anat', 'ds001928_anat', 
           'ds002247_anat', 'ds002001_anat', 'ds002316_anat',
           'ds002076_anat', 'ds002156_anat']

ds_root_paths = glob.glob(orig_face_defaced + '/*_anat*')

processes = set()
max_processes = 16

for data_dir in ds_root_paths:
    print("datadir: ", data_dir)
    dd = data_dir.split('/')[-1]
    save_dd_dir = os.path.join(save_orig_faced_root_dir, dd)
    os.makedirs(save_dd_dir, exist_ok=True)
    ds = glob.glob(data_dir + '/*_defaced.nii*')
    
    for vol in ds:
        vol_name = vol.split('/')[-1].split('.')[0]
        vol_save_path = os.path.join(save_dd_dir, vol_name)
        os.makedirs(vol_save_path, exist_ok=True)
        prefix_pre = vol_save_path
        prefix = os.path.join(prefix_pre, vol_name + '_refaced.nii.gz')
        print("Processing Volume: ", vol)
        print("Save Path: ", prefix)
        processes.add(subprocess.Popen(["singularity","exec", simg_path, "@afni_refacer_run",
                                        "-input", vol,
                                        "-mode_reface_plus",
                                        "-prefix", prefix
                                       ]))
        
        if len(processes) >= max_processes:
            os.wait()
            processes.difference_update([
                p for p in processes if p.poll() is not None])

#     print(ds)
# ds_paths = glob.glob(orig_face_defaced + '*_defaced.nii*')

# ds_paths = glob.glob(orig_data_deface + '/*ds*anat*')

            
# for p in ds_paths:
#     for ex in exclude:
#         if ex not in p:
#             print(p)

#             ds = p.split('/')[-1]
#             os.makedirs(os.path.join(save_root_dir, ds), exist_ok=True)

#             volumes = glob.glob(p + '/*.nii*')
#         #     print(volumes)

#             for vol in volumes:
#                 v = vol.split('/')[-1].split('.')[0]

#                 prefix_pre = os.path.join(save_root_dir, ds, v)
#                 os.makedirs(prefix_pre, exist_ok=True)
#                 prefix = os.path.join(prefix_pre, v + '_refaced.nii.gz')
#                 print("Processing Volume: ", vol)
                
#                 processes.add(subprocess.Popen(["singularity","exec", simg_path, "@afni_refacer_run",
#                                  "-input", vol,
#                                  "-mode_reface_plus",
#                                  "-prefix", prefix
#                                 ]))
                
#                 if len(processes) >= max_processes:
#                     os.wait()
#                     processes.difference_update([
#                         p for p in processes if p.poll() is not None])
            

In [2]:
import os
import glob
import subprocess
import time

orig_root_dir = '/work/01329/poldrack/data/mriqc-net/data'
orig_face_defaced = os.path.join(orig_root_dir, 'masks')

orig_reface_dir = '/work/06850/sbansal6/maverick2/mriqc-shared/refaced'
save_orig_faced_root_dir = os.path.join(orig_reface_dir, 'orig_faced')

simg_path = os.path.join(orig_reface_dir,'afni-latest.simg')
print(simg_path)

# list of datasets already refaced from the original defaced datasets
deface_exclude = [
    'ds002033_anat',
    'ds002149_anat',
    'ds001928_anat', 
    'ds002247_anat',
    'ds002001_anat',
    'ds002316_anat',
    'ds002076_anat',
    'ds002156_anat'
]

# list of datasets already refaced from the orignal faced datasets
# list of datasets already refaced from the orignal faced datasets
face_exclude = [
    'ds000140_anat',
    'ds000119_anat',
    'ds000157_anat',
    'ds002509_anat',
    'ds001650_anat',
    'ds000208_anat',
    'ds001569_anat'
]

face_parallel = [
    'ds000157_anat',
    'ds000205_anat',
    'ds000232_anat',
    'ds001019_anat',
    'ds001393_anat',
    'ds001568_anat',
    'ds001650_anat',
    'ds002572_anat',
    'ds000118_anat',
    'ds000140_anat',
    'ds000159_anat',
    'ds000206_anat',
    'ds000245_anat',
    'ds001037_anat',
    'ds001505_anat',
    'ds001569_anat',
    'ds001900_anat',
    'ds002578_anat'
]

ds_root_paths = glob.glob(orig_face_defaced + '/*_anat*')

processes = set()
max_processes = 16

for data_dir in ds_root_paths:
    print("datadir: ", data_dir)
    dd = data_dir.split('/')[-1]
    if dd not in face_exclude and dd not in face_parallel:
        print(dd)
        save_dd_dir = os.path.join(save_orig_faced_root_dir, dd)
        os.makedirs(save_dd_dir, exist_ok=True)
        ds = glob.glob(data_dir + '/*_defaced.nii*')

        for vol in ds:
            vol_name = vol.split('/')[-1].split('.')[0]
            vol_save_path = os.path.join(save_dd_dir, vol_name)
            os.makedirs(vol_save_path, exist_ok=True)
            prefix_pre = vol_save_path
            prefix = os.path.join(prefix_pre, vol_name + '_refaced.nii.gz')
            print("Processing Volume: ", vol)
            print("Save Path: ", prefix)
            processes.add(subprocess.Popen(["singularity","exec", simg_path, "@afni_refacer_run",
                                            "-input", vol,
                                            "-mode_reface_plus",
                                            "-prefix", prefix
                                           ]))

            if len(processes) >= max_processes:
                os.wait()
                processes.difference_update([
                    p for p in processes if p.poll() is not None])


/work/06850/sbansal6/maverick2/mriqc-shared/refaced/afni-latest.simg
datadir:  /work/01329/poldrack/data/mriqc-net/data/masks/ds000140_anat
datadir:  /work/01329/poldrack/data/mriqc-net/data/masks/ds000119_anat
datadir:  /work/01329/poldrack/data/mriqc-net/data/masks/ds000157_anat
datadir:  /work/01329/poldrack/data/mriqc-net/data/masks/ds002509_anat
datadir:  /work/01329/poldrack/data/mriqc-net/data/masks/ds001650_anat
datadir:  /work/01329/poldrack/data/mriqc-net/data/masks/ds000208_anat
datadir:  /work/01329/poldrack/data/mriqc-net/data/masks/ds001569_anat
datadir:  /work/01329/poldrack/data/mriqc-net/data/masks/ds002572_anat
datadir:  /work/01329/poldrack/data/mriqc-net/data/masks/ds000168_anat
ds000168_anat
Processing Volume:  /work/01329/poldrack/data/mriqc-net/data/masks/ds000168_anat/sub-45_T1w_defaced.nii.gz
Save Path:  /work/06850/sbansal6/maverick2/mriqc-shared/refaced/orig_faced/ds000168_anat/sub-45_T1w_defaced/sub-45_T1w_defaced_refaced.nii.gz
Processing Volume:  /work/013

Processing Volume:  /work/01329/poldrack/data/mriqc-net/data/masks/ds000168_anat/sub-13_T1w_defaced.nii.gz
Save Path:  /work/06850/sbansal6/maverick2/mriqc-shared/refaced/orig_faced/ds000168_anat/sub-13_T1w_defaced/sub-13_T1w_defaced_refaced.nii.gz
Processing Volume:  /work/01329/poldrack/data/mriqc-net/data/masks/ds000168_anat/sub-08_T1w_defaced.nii.gz
Save Path:  /work/06850/sbansal6/maverick2/mriqc-shared/refaced/orig_faced/ds000168_anat/sub-08_T1w_defaced/sub-08_T1w_defaced_refaced.nii.gz
Processing Volume:  /work/01329/poldrack/data/mriqc-net/data/masks/ds000168_anat/sub-26_T1w_defaced.nii.gz
Save Path:  /work/06850/sbansal6/maverick2/mriqc-shared/refaced/orig_faced/ds000168_anat/sub-26_T1w_defaced/sub-26_T1w_defaced_refaced.nii.gz
Processing Volume:  /work/01329/poldrack/data/mriqc-net/data/masks/ds000168_anat/sub-46_T1w_defaced.nii.gz
Save Path:  /work/06850/sbansal6/maverick2/mriqc-shared/refaced/orig_faced/ds000168_anat/sub-46_T1w_defaced/sub-46_T1w_defaced_refaced.nii.gz
data

Processing Volume:  /work/01329/poldrack/data/mriqc-net/data/masks/ds000148_anat/sub-38_acq-axial_T1w_defaced.nii.gz
Save Path:  /work/06850/sbansal6/maverick2/mriqc-shared/refaced/orig_faced/ds000148_anat/sub-38_acq-axial_T1w_defaced/sub-38_acq-axial_T1w_defaced_refaced.nii.gz
Processing Volume:  /work/01329/poldrack/data/mriqc-net/data/masks/ds000148_anat/sub-47_acq-axial_T1w_defaced.nii.gz
Save Path:  /work/06850/sbansal6/maverick2/mriqc-shared/refaced/orig_faced/ds000148_anat/sub-47_acq-axial_T1w_defaced/sub-47_acq-axial_T1w_defaced_refaced.nii.gz
Processing Volume:  /work/01329/poldrack/data/mriqc-net/data/masks/ds000148_anat/sub-21_acq-axial_T1w_defaced.nii.gz
Save Path:  /work/06850/sbansal6/maverick2/mriqc-shared/refaced/orig_faced/ds000148_anat/sub-21_acq-axial_T1w_defaced/sub-21_acq-axial_T1w_defaced_refaced.nii.gz
Processing Volume:  /work/01329/poldrack/data/mriqc-net/data/masks/ds000148_anat/sub-18_acq-axial_T1w_defaced.nii.gz
Save Path:  /work/06850/sbansal6/maverick2/mri

Processing Volume:  /work/01329/poldrack/data/mriqc-net/data/masks/ds000121_anat/sub-10_T1w_defaced.nii.gz
Save Path:  /work/06850/sbansal6/maverick2/mriqc-shared/refaced/orig_faced/ds000121_anat/sub-10_T1w_defaced/sub-10_T1w_defaced_refaced.nii.gz
Processing Volume:  /work/01329/poldrack/data/mriqc-net/data/masks/ds000121_anat/sub-01_T1w_defaced.nii.gz
Save Path:  /work/06850/sbansal6/maverick2/mriqc-shared/refaced/orig_faced/ds000121_anat/sub-01_T1w_defaced/sub-01_T1w_defaced_refaced.nii.gz
Processing Volume:  /work/01329/poldrack/data/mriqc-net/data/masks/ds000121_anat/sub-16_T1w_defaced.nii.gz
Save Path:  /work/06850/sbansal6/maverick2/mriqc-shared/refaced/orig_faced/ds000121_anat/sub-16_T1w_defaced/sub-16_T1w_defaced_refaced.nii.gz
Processing Volume:  /work/01329/poldrack/data/mriqc-net/data/masks/ds000121_anat/sub-21_T1w_defaced.nii.gz
Save Path:  /work/06850/sbansal6/maverick2/mriqc-shared/refaced/orig_faced/ds000121_anat/sub-21_T1w_defaced/sub-21_T1w_defaced_refaced.nii.gz
Proc

Processing Volume:  /work/01329/poldrack/data/mriqc-net/data/masks/ds000164_anat/sub-017_T1w_defaced.nii.gz
Save Path:  /work/06850/sbansal6/maverick2/mriqc-shared/refaced/orig_faced/ds000164_anat/sub-017_T1w_defaced/sub-017_T1w_defaced_refaced.nii.gz
Processing Volume:  /work/01329/poldrack/data/mriqc-net/data/masks/ds000164_anat/sub-022_T1w_defaced.nii.gz
Save Path:  /work/06850/sbansal6/maverick2/mriqc-shared/refaced/orig_faced/ds000164_anat/sub-022_T1w_defaced/sub-022_T1w_defaced_refaced.nii.gz
Processing Volume:  /work/01329/poldrack/data/mriqc-net/data/masks/ds000164_anat/sub-013_T1w_defaced.nii.gz
Save Path:  /work/06850/sbansal6/maverick2/mriqc-shared/refaced/orig_faced/ds000164_anat/sub-013_T1w_defaced/sub-013_T1w_defaced_refaced.nii.gz
Processing Volume:  /work/01329/poldrack/data/mriqc-net/data/masks/ds000164_anat/sub-014_T1w_defaced.nii.gz
Save Path:  /work/06850/sbansal6/maverick2/mriqc-shared/refaced/orig_faced/ds000164_anat/sub-014_T1w_defaced/sub-014_T1w_defaced_refaced

...

Processing Volume:  /work/01329/poldrack/data/mriqc-net/data/masks/ds000109_anat/sub-14_T1w_defaced.nii
Save Path:  /work/06850/sbansal6/maverick2/mriqc-shared/refaced/orig_faced/ds000109_anat/sub-14_T1w_defaced/sub-14_T1w_defaced_refaced.nii.gz
Processing Volume:  /work/01329/poldrack/data/mriqc-net/data/masks/ds000109_anat/sub-29_T1w_defaced.nii
Save Path:  /work/06850/sbansal6/maverick2/mriqc-shared/refaced/orig_faced/ds000109_anat/sub-29_T1w_defaced/sub-29_T1w_defaced_refaced.nii.gz
Processing Volume:  /work/01329/poldrack/data/mriqc-net/data/masks/ds000109_anat/sub-48_T1w_defaced.nii
Save Path:  /work/06850/sbansal6/maverick2/mriqc-shared/refaced/orig_faced/ds000109_anat/sub-48_T1w_defaced/sub-48_T1w_defaced_refaced.nii.gz
Processing Volume:  /work/01329/poldrack/data/mriqc-net/data/masks/ds000109_anat/sub-25_T1w_defaced.nii
Save Path:  /work/06850/sbansal6/maverick2/mriqc-shared/refaced/orig_faced/ds000109_anat/sub-25_T1w_defaced/sub-25_T1w_defaced_refaced.nii.gz
Processing Volum

Processing Volume:  /work/01329/poldrack/data/mriqc-net/data/masks/ds000009_anat/sub-19_T1w_defaced.nii.gz
Save Path:  /work/06850/sbansal6/maverick2/mriqc-shared/refaced/orig_faced/ds000009_anat/sub-19_T1w_defaced/sub-19_T1w_defaced_refaced.nii.gz
Processing Volume:  /work/01329/poldrack/data/mriqc-net/data/masks/ds000009_anat/sub-08_T1w_defaced.nii.gz
Save Path:  /work/06850/sbansal6/maverick2/mriqc-shared/refaced/orig_faced/ds000009_anat/sub-08_T1w_defaced/sub-08_T1w_defaced_refaced.nii.gz
Processing Volume:  /work/01329/poldrack/data/mriqc-net/data/masks/ds000009_anat/sub-26_T1w_defaced.nii.gz
Save Path:  /work/06850/sbansal6/maverick2/mriqc-shared/refaced/orig_faced/ds000009_anat/sub-26_T1w_defaced/sub-26_T1w_defaced_refaced.nii.gz
datadir:  /work/01329/poldrack/data/mriqc-net/data/masks/ds001583_anat
ds001583_anat
Processing Volume:  /work/01329/poldrack/data/mriqc-net/data/masks/ds001583_anat/sub-SS47_T1w_defaced.nii.gz
Save Path:  /work/06850/sbansal6/maverick2/mriqc-shared/ref

Processing Volume:  /work/01329/poldrack/data/mriqc-net/data/masks/ds001996_anat/sub-26_T1w_defaced.nii.gz
Save Path:  /work/06850/sbansal6/maverick2/mriqc-shared/refaced/orig_faced/ds001996_anat/sub-26_T1w_defaced/sub-26_T1w_defaced_refaced.nii.gz
datadir:  /work/01329/poldrack/data/mriqc-net/data/masks/ds000031_anat
ds000031_anat
Processing Volume:  /work/01329/poldrack/data/mriqc-net/data/masks/ds000031_anat/sub-01_ses-024_run-001_T1w_defaced.nii.gz
Save Path:  /work/06850/sbansal6/maverick2/mriqc-shared/refaced/orig_faced/ds000031_anat/sub-01_ses-024_run-001_T1w_defaced/sub-01_ses-024_run-001_T1w_defaced_refaced.nii.gz
Processing Volume:  /work/01329/poldrack/data/mriqc-net/data/masks/ds000031_anat/sub-01_ses-036_run-001_T1w_defaced.nii.gz
Save Path:  /work/06850/sbansal6/maverick2/mriqc-shared/refaced/orig_faced/ds000031_anat/sub-01_ses-036_run-001_T1w_defaced/sub-01_ses-036_run-001_T1w_defaced_refaced.nii.gz
Processing Volume:  /work/01329/poldrack/data/mriqc-net/data/masks/ds000

Processing Volume:  /work/01329/poldrack/data/mriqc-net/data/masks/ds000149_anat/sub-04_T1w_defaced.nii.gz
Save Path:  /work/06850/sbansal6/maverick2/mriqc-shared/refaced/orig_faced/ds000149_anat/sub-04_T1w_defaced/sub-04_T1w_defaced_refaced.nii.gz
Processing Volume:  /work/01329/poldrack/data/mriqc-net/data/masks/ds000149_anat/sub-10_T1w_defaced.nii.gz
Save Path:  /work/06850/sbansal6/maverick2/mriqc-shared/refaced/orig_faced/ds000149_anat/sub-10_T1w_defaced/sub-10_T1w_defaced_refaced.nii.gz
Processing Volume:  /work/01329/poldrack/data/mriqc-net/data/masks/ds000149_anat/sub-01_T1w_defaced.nii.gz
Save Path:  /work/06850/sbansal6/maverick2/mriqc-shared/refaced/orig_faced/ds000149_anat/sub-01_T1w_defaced/sub-01_T1w_defaced_refaced.nii.gz
Processing Volume:  /work/01329/poldrack/data/mriqc-net/data/masks/ds000149_anat/sub-02_T1w_defaced.nii.gz
Save Path:  /work/06850/sbansal6/maverick2/mriqc-shared/refaced/orig_faced/ds000149_anat/sub-02_T1w_defaced/sub-02_T1w_defaced_refaced.nii.gz
Proc

Processing Volume:  /work/01329/poldrack/data/mriqc-net/data/masks/ds000213_anat/sub-13_T1map_defaced.nii
Save Path:  /work/06850/sbansal6/maverick2/mriqc-shared/refaced/orig_faced/ds000213_anat/sub-13_T1map_defaced/sub-13_T1map_defaced_refaced.nii.gz
Processing Volume:  /work/01329/poldrack/data/mriqc-net/data/masks/ds000213_anat/sub-20_T1map_defaced.nii
Save Path:  /work/06850/sbansal6/maverick2/mriqc-shared/refaced/orig_faced/ds000213_anat/sub-20_T1map_defaced/sub-20_T1map_defaced_refaced.nii.gz
Processing Volume:  /work/01329/poldrack/data/mriqc-net/data/masks/ds000213_anat/sub-04_T1map_defaced.nii
Save Path:  /work/06850/sbansal6/maverick2/mriqc-shared/refaced/orig_faced/ds000213_anat/sub-04_T1map_defaced/sub-04_T1map_defaced_refaced.nii.gz
Processing Volume:  /work/01329/poldrack/data/mriqc-net/data/masks/ds000213_anat/sub-22_T1map_defaced.nii
Save Path:  /work/06850/sbansal6/maverick2/mriqc-shared/refaced/orig_faced/ds000213_anat/sub-22_T1map_defaced/sub-22_T1map_defaced_refaced

In [4]:
import matplotlib 
# matplotlib.use('Agg')
import os, sys
sys.path.append('../defacing')
import subprocess

from preprocessing.normalization import clip, standardize, normalize
from preprocessing.conform import conform_data
from helpers.utils import load_vol, save_vol, is_gz_file
import numpy as np
import nibabel as nib
from glob import glob
import matplotlib.pyplot as plt
import tempfile


exclude_face_refaced = ['ds000140_anat',
           'ds000119_anat',
           'ds000157_anat',
           'ds002509_anat'
          ]
    
    
orig_root_dir = '/work/01329/poldrack/data/mriqc-net/data'
orig_data_face = os.path.join(orig_root_dir, 'face/T1w')
orig_data_mask = os.path.join(orig_root_dir, 'masks')

orig_data_deface = os.path.join(orig_root_dir, 'defaced')

save_root_dir = '/work/06850/sbansal6/maverick2/mriqc-shared/'

orig_reface_dir = '/work/06850/sbansal6/maverick2/mriqc-shared/refaced'
save_orig_faced_root_dir = os.path.join(orig_reface_dir, 'orig_faced')


save_preprocessing_face = os.path.join(save_root_dir, 'preprocessing/face')
save_conformed_face = os.path.join(save_root_dir, 'conformed/face')

save_preprocessing_deface = os.path.join(save_root_dir, 'preprocessing/deface')
save_conformed_deface = os.path.join(save_root_dir, 'conformed/deface')

save_preprocessing_face_defaced = os.path.join(save_root_dir, 'preprocessing/face_defaced')
save_conformed_face_defaced = os.path.join(save_root_dir, 'conformed/face_defaced')

save_preprocessing_face_refaced = os.path.join(save_root_dir, 'preprocessing/face_refaced')
save_conformed_face_refaced = os.path.join(save_root_dir, 'conformed/face_refaced')

# os.makedirs(save_preprocessing_face, exist_ok=True)
os.makedirs(save_preprocessing_deface, exist_ok=True)
# os.makedirs(save_conformed_face, exist_ok=True)
os.makedirs(save_conformed_deface, exist_ok=True)

conform_size = (64, 64, 64)
conform_zoom = (4., 4., 4.)


def preprocess(orig_vol_pth, conform_pth, preprocess_pth, DS=None, mask_path=None, debug=False):
    """
    """
    
    filename = orig_vol_pth.split('/')[-1]
    volume, affine, _ = load_vol(orig_vol_pth)
    
    # Preprocessing
    volume = clip(volume, q=90)
    volume = normalize(volume)
    volume = standardize(volume)
    
    # 
    save_preprocessing_path = os.path.join(preprocess_pth, filename)
    save_conformed_path = os.path.join(conform_pth, filename)
    
    print("save_preprocessing_path: ", save_preprocessing_path)
    print("save_conformed_path: ", save_conformed_path)
    
    save_vol(save_preprocessing_path, volume, affine)

    def _plot(data):
        f, axarr = plt.subplots(8, 8, figsize=(12, 12))
        for i in range(8):
            for j in range(8):
                axarr[i, j].imshow(np.rot90(data[:, :, j + 8*i], 1))

        plt.show()
        
        
#         plt.subplot(1, 3, 1)
#         plt.imshow(np.rot90(np.mean(data, axis=0)))
#         plt.subplot(1, 3, 2)
#         plt.imshow(np.rot90(np.mean(data, axis=1)))
#         plt.subplot(1, 3, 3)
#         plt.imshow(np.rot90(np.mean(data, axis=2)))
#         plt.show()
        
    conform_data(save_preprocessing_path, 
                 out_file=save_conformed_path, 
                 out_size=conform_size, 
                 out_zooms=conform_zoom)

    if debug: _plot(load_vol(save_conformed_path)[0])
    
    if mask_path and DS:
        mask = np.array(nib.load(mask_path).dataobj)
        masked_volume = volume*mask

        save_mpreprocessing_path = os.path.join(save_preprocessing_deface, DS, filename)
        save_mconformed_path = os.path.join(save_conformed_deface, DS, filename)
        
        print("save_deface_preprocessing_path: ", save_mpreprocessing_path)
        print("save_deface_conformed_path: ", save_mconformed_path)
        
#         os.makedirs(save_mpreprocessing_path, exist_ok=True)
        os.makedirs(os.path.dirname(save_mconformed_path), exist_ok=True)
    
#         save_mpreprocessing_path = os
        
        save_vol(save_mpreprocessing_path, masked_volume, affine)

        conform_data(save_mpreprocessing_path, 
                 out_file=save_mconformed_path, 
                 out_size=conform_size, 
                 out_zooms=conform_zoom)        
        
        return save_conformed_path, save_mconformed_path

    return save_conformed_path


# for path in glob(orig_data_face + "/ds000140_anat/*.nii*"):
# #     try:
        
#     vol_name = path.split("/")[-1]
#     DS = path.split("/")[-2]

#     # directories for saving preprocessed and conformed volumes
#     ds_save_conform_path = os.path.join(save_conformed_face, DS)
#     ds_save_preprocess_path = os.path.join(save_preprocessing_face, DS)

#     # Get the mask path
#     mask_path = glob(os.path.join(orig_data_mask, DS, vol_name.split('.')[0] + "*_mask*"))[0]

#     print("Mask_path", mask_path)

#     if not os.path.exists(ds_save_conform_path):
#         os.makedirs(ds_save_conform_path)

#     if not os.path.exists(ds_save_preprocess_path):
#         os.makedirs(ds_save_preprocess_path)

#     # Check if volume is a proper gunzipped
#     print(os.path.splitext(path)[1])
#     print("Gzipped: ", is_gz_file(path))
#     if os.path.splitext(path)[1] == ".gz" and not is_gz_file(path):
#         rename_file = os.path.splitext(vol_name)[0]
#         fixed_gz_tmp = os.path.join(save_conformed_face, rename_file)
#         print("Fixed GZIP File: ", fixed_gz_tmp)
#         subprocess.call(["cp", path, fixed_gz_tmp])

#         print(preprocess(fixed_gz_tmp,
#                          ds_save_conform_path,
#                          ds_save_preprocess_path,
#                          mask_path = mask_path,
#                          DS=DS
#                         ))
#         os.remove(fixed_gz_tmp)

#     else:
#         print(preprocess(path,
#                          ds_save_conform_path,
#                          ds_save_preprocess_path,
#                          mask_path = mask_path,
#                          DS=DS
#                         ))
#     except:
#         print("Preprocessing incomplete. Exception occurred.")
#         pass


# for ds in include:
#     volumes = glob(os.path.join(save_orig_faced_root_dir, ds) + '/*defaced*/*refaced.nii*')
    
    
for path in glob(save_orig_faced_root_dir + "/*/*defaced*/*refaced.nii*"):
    try:

        vol_name = path.split("/")[-1]
        DS = path.split("/")[-3]
        if DS not in exclude_face_refaced:
            print(vol_name, DS)

            # directories for saving preprocessed and conformed volumes
            ds_save_conform_path = os.path.join(save_conformed_face_refaced, DS)
            ds_save_preprocess_path = os.path.join(save_preprocessing_face_refaced, DS)

            os.makedirs(ds_save_conform_path, exist_ok=True)
            os.makedirs(ds_save_preprocess_path, exist_ok=True)

            # Check if volume is a proper gunzipped
            if not os.path.splitext(path)[1] == ".gz" and is_gz_file(path):
                rename_file = os.path.splitext(vol_name)[0]
                fixed_gz_tmp = os.path.join(save_conformed_face_refaced, rename_file)
                print(fixed_gz_tmp)
                subprocess.call(["cp", path, fixed_gz_tmp])

                print(preprocess(fixed_gz_tmp,
                                 ds_save_conform_path,
                                 ds_save_preprocess_path,
                                 DS=DS
                                ))
                os.remove(fixed_gz_tmp)

            else:
                print(preprocess(path,
                                 ds_save_conform_path,
                                 ds_save_preprocess_path,
                                 DS=DS
                                ))
                
    except:
        print("Preprocessing incomplete. Exception occurred.")
        pass



sub-5012_T1w_defaced_refaced.nii.gz ds001650_anat
save_preprocessing_path:  /work/06850/sbansal6/maverick2/mriqc-shared/preprocessing/face_refaced/ds001650_anat/sub-5012_T1w_defaced_refaced.nii.gz
save_conformed_path:  /work/06850/sbansal6/maverick2/mriqc-shared/conformed/face_refaced/ds001650_anat/sub-5012_T1w_defaced_refaced.nii.gz
[22.  26.5 39.5]
/work/06850/sbansal6/maverick2/mriqc-shared/conformed/face_refaced/ds001650_anat/sub-5012_T1w_defaced_refaced.nii.gz
sub-5008_T1w_defaced_refaced.nii.gz ds001650_anat
save_preprocessing_path:  /work/06850/sbansal6/maverick2/mriqc-shared/preprocessing/face_refaced/ds001650_anat/sub-5008_T1w_defaced_refaced.nii.gz
save_conformed_path:  /work/06850/sbansal6/maverick2/mriqc-shared/conformed/face_refaced/ds001650_anat/sub-5008_T1w_defaced_refaced.nii.gz
[22.  24.5 36.5]
/work/06850/sbansal6/maverick2/mriqc-shared/conformed/face_refaced/ds001650_anat/sub-5008_T1w_defaced_refaced.nii.gz
sub-3798_T1w_defaced_refaced.nii.gz ds001650_anat
save_prepr

save_preprocessing_path:  /work/06850/sbansal6/maverick2/mriqc-shared/preprocessing/face_refaced/ds001650_anat/sub-5027_T1w_defaced_refaced.nii.gz
save_conformed_path:  /work/06850/sbansal6/maverick2/mriqc-shared/conformed/face_refaced/ds001650_anat/sub-5027_T1w_defaced_refaced.nii.gz
[22.         21.5        26.00000751]
/work/06850/sbansal6/maverick2/mriqc-shared/conformed/face_refaced/ds001650_anat/sub-5027_T1w_defaced_refaced.nii.gz
sub-3201_T1w_defaced_refaced.nii.gz ds001650_anat
save_preprocessing_path:  /work/06850/sbansal6/maverick2/mriqc-shared/preprocessing/face_refaced/ds001650_anat/sub-3201_T1w_defaced_refaced.nii.gz
save_conformed_path:  /work/06850/sbansal6/maverick2/mriqc-shared/conformed/face_refaced/ds001650_anat/sub-3201_T1w_defaced_refaced.nii.gz
[22.73684754 14.5        26.        ]
/work/06850/sbansal6/maverick2/mriqc-shared/conformed/face_refaced/ds001650_anat/sub-3201_T1w_defaced_refaced.nii.gz
sub-3441_T1w_defaced_refaced.nii.gz ds001650_anat
save_preprocessing

save_preprocessing_path:  /work/06850/sbansal6/maverick2/mriqc-shared/preprocessing/face_refaced/ds001650_anat/sub-5014_T1w_defaced_refaced.nii.gz
save_conformed_path:  /work/06850/sbansal6/maverick2/mriqc-shared/conformed/face_refaced/ds001650_anat/sub-5014_T1w_defaced_refaced.nii.gz
[22.  29.  31.5]
/work/06850/sbansal6/maverick2/mriqc-shared/conformed/face_refaced/ds001650_anat/sub-5014_T1w_defaced_refaced.nii.gz
sub-5026_T1w_defaced_refaced.nii.gz ds001650_anat
save_preprocessing_path:  /work/06850/sbansal6/maverick2/mriqc-shared/preprocessing/face_refaced/ds001650_anat/sub-5026_T1w_defaced_refaced.nii.gz
save_conformed_path:  /work/06850/sbansal6/maverick2/mriqc-shared/conformed/face_refaced/ds001650_anat/sub-5026_T1w_defaced_refaced.nii.gz
[21.99998498 29.         38.5       ]
/work/06850/sbansal6/maverick2/mriqc-shared/conformed/face_refaced/ds001650_anat/sub-5026_T1w_defaced_refaced.nii.gz
sub-3256_T1w_defaced_refaced.nii.gz ds001650_anat
save_preprocessing_path:  /work/06850/s

save_preprocessing_path:  /work/06850/sbansal6/maverick2/mriqc-shared/preprocessing/face_refaced/ds001650_anat/sub-3733_T1w_defaced_refaced.nii.gz
save_conformed_path:  /work/06850/sbansal6/maverick2/mriqc-shared/conformed/face_refaced/ds001650_anat/sub-3733_T1w_defaced_refaced.nii.gz
[22.73686452 24.5        26.        ]
/work/06850/sbansal6/maverick2/mriqc-shared/conformed/face_refaced/ds001650_anat/sub-3733_T1w_defaced_refaced.nii.gz
sub-3372_T1w_defaced_refaced.nii.gz ds001650_anat
save_preprocessing_path:  /work/06850/sbansal6/maverick2/mriqc-shared/preprocessing/face_refaced/ds001650_anat/sub-3372_T1w_defaced_refaced.nii.gz
save_conformed_path:  /work/06850/sbansal6/maverick2/mriqc-shared/conformed/face_refaced/ds001650_anat/sub-3372_T1w_defaced_refaced.nii.gz
[22.73684754 21.         26.        ]
/work/06850/sbansal6/maverick2/mriqc-shared/conformed/face_refaced/ds001650_anat/sub-3372_T1w_defaced_refaced.nii.gz
sub-5001_T1w_defaced_refaced.nii.gz ds001650_anat
save_preprocessing

[-2. -2. 46.]
/work/06850/sbansal6/maverick2/mriqc-shared/conformed/face_refaced/ds000208_anat/sub-58_ses-3_T1w_defaced_refaced.nii.gz
sub-24_ses-1_T1w_defaced_refaced.nii.gz ds000208_anat
save_preprocessing_path:  /work/06850/sbansal6/maverick2/mriqc-shared/preprocessing/face_refaced/ds000208_anat/sub-24_ses-1_T1w_defaced_refaced.nii.gz
save_conformed_path:  /work/06850/sbansal6/maverick2/mriqc-shared/conformed/face_refaced/ds000208_anat/sub-24_ses-1_T1w_defaced_refaced.nii.gz
[-2. -2. 46.]
/work/06850/sbansal6/maverick2/mriqc-shared/conformed/face_refaced/ds000208_anat/sub-24_ses-1_T1w_defaced_refaced.nii.gz
sub-04_ses-1_T1w_defaced_refaced.nii.gz ds000208_anat
save_preprocessing_path:  /work/06850/sbansal6/maverick2/mriqc-shared/preprocessing/face_refaced/ds000208_anat/sub-04_ses-1_T1w_defaced_refaced.nii.gz
save_conformed_path:  /work/06850/sbansal6/maverick2/mriqc-shared/conformed/face_refaced/ds000208_anat/sub-04_ses-1_T1w_defaced_refaced.nii.gz
[-2. -2. 46.]
/work/06850/sbansal6

...

[85.0002475  -3.3749802  -3.37500158]
/work/06850/sbansal6/maverick2/mriqc-shared/conformed/face_refaced/ds000206_anat/sub-THP0002_ses-THP0002DART1_acq-T130_run-1302_T1w_defaced_refaced.nii.gz
sub-THP0004_ses-THP0004JHU1_acq-T130_run-301_T1w_defaced_refaced.nii.gz ds000206_anat
save_preprocessing_path:  /work/06850/sbansal6/maverick2/mriqc-shared/preprocessing/face_refaced/ds000206_anat/sub-THP0004_ses-THP0004JHU1_acq-T130_run-301_T1w_defaced_refaced.nii.gz
save_conformed_path:  /work/06850/sbansal6/maverick2/mriqc-shared/conformed/face_refaced/ds000206_anat/sub-THP0004_ses-THP0004JHU1_acq-T130_run-301_T1w_defaced_refaced.nii.gz
[ 5.5999514  32.54440934  5.59996449]
/work/06850/sbansal6/maverick2/mriqc-shared/conformed/face_refaced/ds000206_anat/sub-THP0004_ses-THP0004JHU1_acq-T130_run-301_T1w_defaced_refaced.nii.gz
sub-THP0003_ses-THP0003IOWA3_acq-T130_run-003_T1wb_defaced_refaced.nii.gz ds000206_anat
save_preprocessing_path:  /work/06850/sbansal6/maverick2/mriqc-shared/preprocessing/

[-13.61702128  -5.45518102 -13.61702128]
/work/06850/sbansal6/maverick2/mriqc-shared/conformed/face_refaced/ds000206_anat/sub-THP0004_ses-THP0004IOWA2_acq-T130_run-002_T1wb_defaced_refaced.nii.gz
sub-THP0003_ses-THP0003JHU1_acq-T130_run-401_T1wb_defaced_refaced.nii.gz ds000206_anat
save_preprocessing_path:  /work/06850/sbansal6/maverick2/mriqc-shared/preprocessing/face_refaced/ds000206_anat/sub-THP0003_ses-THP0003JHU1_acq-T130_run-401_T1wb_defaced_refaced.nii.gz
save_conformed_path:  /work/06850/sbansal6/maverick2/mriqc-shared/conformed/face_refaced/ds000206_anat/sub-THP0003_ses-THP0003JHU1_acq-T130_run-401_T1wb_defaced_refaced.nii.gz
[ 5.5999514  32.54527827  5.60001682]
/work/06850/sbansal6/maverick2/mriqc-shared/conformed/face_refaced/ds000206_anat/sub-THP0003_ses-THP0003JHU1_acq-T130_run-401_T1wb_defaced_refaced.nii.gz
sub-THP0002_ses-THP0002DART1_acq-T130_run-1202_T1w_defaced_refaced.nii.gz ds000206_anat
save_preprocessing_path:  /work/06850/sbansal6/maverick2/mriqc-shared/preproc

[85.0002475  -3.3749802  -3.37500158]
/work/06850/sbansal6/maverick2/mriqc-shared/conformed/face_refaced/ds000206_anat/sub-THP0002_ses-THP0002DART1_acq-T130_run-1301_T1w_defaced_refaced.nii.gz
sub-THP0002_ses-THP0002DART1_acq-T130_run-1201_T1w_defaced_refaced.nii.gz ds000206_anat
save_preprocessing_path:  /work/06850/sbansal6/maverick2/mriqc-shared/preprocessing/face_refaced/ds000206_anat/sub-THP0002_ses-THP0002DART1_acq-T130_run-1201_T1w_defaced_refaced.nii.gz
save_conformed_path:  /work/06850/sbansal6/maverick2/mriqc-shared/conformed/face_refaced/ds000206_anat/sub-THP0002_ses-THP0002DART1_acq-T130_run-1201_T1w_defaced_refaced.nii.gz
[85.0002475  -3.3749802  -3.37500158]
/work/06850/sbansal6/maverick2/mriqc-shared/conformed/face_refaced/ds000206_anat/sub-THP0002_ses-THP0002DART1_acq-T130_run-1201_T1w_defaced_refaced.nii.gz
sub-THP0003_ses-THP0003UW1_acq-T130_run-901_T1wc_defaced_refaced.nii.gz ds000206_anat
save_preprocessing_path:  /work/06850/sbansal6/maverick2/mriqc-shared/preproce

save_preprocessing_path:  /work/06850/sbansal6/maverick2/mriqc-shared/preprocessing/face_refaced/ds000213_anat/sub-04_T1map_defaced_refaced.nii.gz
save_conformed_path:  /work/06850/sbansal6/maverick2/mriqc-shared/conformed/face_refaced/ds000213_anat/sub-04_T1map_defaced_refaced.nii.gz
[38.     1.024  1.024]
/work/06850/sbansal6/maverick2/mriqc-shared/conformed/face_refaced/ds000213_anat/sub-04_T1map_defaced_refaced.nii.gz
sub-16_T1map_defaced_refaced.nii.gz ds000213_anat
save_preprocessing_path:  /work/06850/sbansal6/maverick2/mriqc-shared/preprocessing/face_refaced/ds000213_anat/sub-16_T1map_defaced_refaced.nii.gz
save_conformed_path:  /work/06850/sbansal6/maverick2/mriqc-shared/conformed/face_refaced/ds000213_anat/sub-16_T1map_defaced_refaced.nii.gz
[38.00000751  1.02399213  1.024     ]
/work/06850/sbansal6/maverick2/mriqc-shared/conformed/face_refaced/ds000213_anat/sub-16_T1map_defaced_refaced.nii.gz
sub-08_T1map_defaced_refaced.nii.gz ds000213_anat
save_preprocessing_path:  /work/0

[38.00000751 -2.         -2.        ]
/work/06850/sbansal6/maverick2/mriqc-shared/conformed/face_refaced/ds000159_anat/sub-46_T1w_defaced_refaced.nii.gz
sub-38_T1w_defaced_refaced.nii.gz ds000159_anat
save_preprocessing_path:  /work/06850/sbansal6/maverick2/mriqc-shared/preprocessing/face_refaced/ds000159_anat/sub-38_T1w_defaced_refaced.nii.gz
save_conformed_path:  /work/06850/sbansal6/maverick2/mriqc-shared/conformed/face_refaced/ds000159_anat/sub-38_T1w_defaced_refaced.nii.gz
[38. -2. -2.]
/work/06850/sbansal6/maverick2/mriqc-shared/conformed/face_refaced/ds000159_anat/sub-38_T1w_defaced_refaced.nii.gz
sub-43_T1w_defaced_refaced.nii.gz ds000159_anat
save_preprocessing_path:  /work/06850/sbansal6/maverick2/mriqc-shared/preprocessing/face_refaced/ds000159_anat/sub-43_T1w_defaced_refaced.nii.gz
save_conformed_path:  /work/06850/sbansal6/maverick2/mriqc-shared/conformed/face_refaced/ds000159_anat/sub-43_T1w_defaced_refaced.nii.gz
[38.00000751 -1.99999249 -2.        ]
/work/06850/sbansal6

In [ ]:
orig_root_dir = '/work/01329/poldrack/data/mriqc-net/data'
orig_data_face = os.path.join(orig_root_dir, 'face/T1w')
# orig_data_mask = os.path.join(orig_root_dir, 'masks')

orig_data_deface = os.path.join(orig_root_dir, 'defaced')

save_root_dir = '/work/06850/sbansal6/maverick2/mriqc-shared/'

# save_preprocessing_face = os.path.join(save_root_dir, 'preprocessing/face')
save_conformed_face = os.path.join(save_root_dir, 'conformed/face')

save_preprocessing_deface = os.path.join(save_root_dir, 'preprocessing/deface')
save_conformed_deface = os.path.join(save_root_dir, 'conformed/deface')

save_conformed_face_defaced = os.path.join(save_root_dir, 'conformed/face_defaced')
# os.makedirs(save_preprocessing_face, exist_ok=True)
# os.makedirs(save_preprocessing_deface, exist_ok=True)
# os.makedirs(save_conformed_face, exist_ok=True)
# os.makedirs(save_conformed_deface, exist_ok=True)


def checkNonConformed(orig_path, save_path):

    conform = []
    orig = []

    for path in glob(save_path + "/*/*.nii*"):
        tempname = path.split("/")[-1]
        ds = path.split("/")[-2]
        conform.append(ds + "/" + tempname)

    print("Total Conformed: ", len(conform))

    for path in glob(orig_path + "/*/*.nii*"):
        tempname = path.split("/")[-1]
        ds = path.split("/")[-2]
        orig.append(ds + "/" + tempname)

    print("Total Original: ", len(orig))

    print("Total not conformed: ", len(orig) - len(conform))

    count = 0
    for f in orig:
        exists = False
        for fc in conform:
            if fc in f:
                exists = True
        if not exists:
            count += 1
            print("Not conformed file: ", f)
            

            
checkNonConformed(orig_data_face, save_conformed_face)


In [ ]:
face_C = []
face_O = []

for path in glob(save_data_face + "/*/*.nii*"):
    tempname = path.split("/")[-1]
    ds = path.split("/")[-2]
    face_C.append(ds + '/' + tempname)

print(len(face_C))
# print(face_C)


for path in glob(orig_data_face + "/*/*.nii*"):
    tempname = path.split("/")[-1]
    ds = path.split("/")[-2]
    face_O.append(ds + '/' + tempname)

print(len(face_O))
# print(face_O)

count = 0
for f in face_O:
    exists = False
    for fc in face_C:
        if fc in f:
            exists = True
    if not exists:
        count += 1
        print(f)
print(count)

## Generate n-fold CV Dataset

In [6]:
import os, sys
sys.path.append("..")
import numpy as np
import nibabel as nb
from glob import glob
from pathlib import Path
from shutil import *
import subprocess
from operator import itemgetter
from sklearn.model_selection import KFold
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import train_test_split
import pandas as pd
from random import shuffle


face_path = "/work/06850/sbansal6/maverick2/mriqc-shared/conformed/face"
deface_path = "/work/06850/sbansal6/maverick2/mriqc-shared/conformed/deface"
face_defaced_path = "/work/06850/sbansal6/maverick2/mriqc-shared/conformed/face_defaced"
face_refaced_path = "/work/06850/sbansal6/maverick2/mriqc-shared/conformed/face_refaced"

paths_d = []
paths_f = []
labels_d = []
labels_f = []

for path in glob(face_defaced_path + "/*/*.nii*"):
    DS = path.split('/')[-2]
    paths_d.append(path)
    labels_d.append(0)
    
# for path in glob(deface_path + "/*/*.nii*"):
#     paths.append(path)
#     labels.append(0)

for path in glob(face_refaced_path + "/*/*.nii*"):
    DS = path.split('/')[-2]
    paths_f.append(path)
    labels_f.append(1)
    
# for path in glob(face_path + "/*/*.nii*"):
#     DS = path.split('/')[-2]
#     paths_f.append(path)
#     labels_f.append(1)
    
print(len(paths_f), len(paths_d))
print(len(labels_f), len(labels_d))


percentage = [25, 50, 75, 100]
percentage = [100]

for per in percentage:
    
    SPLITS = 15
    
#     print(len(paths))
    ind = int(len(paths_f)*per/100)
    shuffle(paths_d)
    shuffle(paths_f)
    temp_p = paths_d[:ind] + paths_f[:ind]
    temp_l = labels_d[:ind] + labels_f[:ind]
    
    save_path = "/work/06850/sbansal6/maverick2/mriqc-shared/experiment_faced_refaced/exp_face_refaced/exp_add_datasets/csv_F15_P{}".format(per)
    os.makedirs(save_path, exist_ok=True)
    
    df = pd.DataFrame()
    df["X"] = temp_p
    df["Y"] = temp_l
    df.to_csv(os.path.join(save_path, "all.csv"))
    
    SPLITS = 15
    skf = StratifiedKFold(n_splits=SPLITS)
    fold_no = 1
    
    for train_index, test_index in skf.split(temp_p, temp_l):
        out_path = os.path.join(save_path, "train_test_fold_{}/csv/".format(fold_no))
        
        if not os.path.exists(out_path):
            os.makedirs(out_path)
            
        image_train, image_test = (
            itemgetter(*train_index)(temp_p),
            itemgetter(*test_index)(temp_p),
        )
        
        label_train, label_test = (
            itemgetter(*train_index)(temp_l),
            itemgetter(*test_index)(temp_l),
        )

        train_data = {"X": image_train , "Y": label_train}
        df_train = pd.DataFrame(train_data)
        df_train.to_csv(os.path.join(out_path, "training.csv"), index=False)
        
        validation_data = {"X": image_test, "Y": label_test}
        df_validation = pd.DataFrame(validation_data)
        df_validation.to_csv(os.path.join(out_path, "validation.csv"), index=False)
        
        fold_no += 1
        
    
    
# df = pd.DataFrame()
# df["X"] = paths
# df["Y"] = labels
# df.to_csv(os.path.join(save_path, "all.csv"))

# SPLITS = 15
# skf = StratifiedKFold(n_splits=SPLITS)
# fold_no = 1

# for train_index, test_index in skf.split(paths, labels):
#     out_path = save_path + "/train_test_fold_{}/csv/".format(fold_no)

#     if not os.path.exists(out_path):
#         os.makedirs(out_path)

#     image_train, image_test = (
#         itemgetter(*train_index)(paths),
#         itemgetter(*test_index)(paths),
#     )
#     label_train, label_test = (
#         itemgetter(*train_index)(labels),
#         itemgetter(*test_index)(labels),
#     )

#     # image_train = [os.path.join(data_path, 'sub-' + str(pth) + '_T1w.nii.gz') for pth in image_train]
#     train_data = {"X": image_train, "Y": label_train}
#     df_train = pd.DataFrame(train_data)
#     df_train.to_csv(os.path.join(out_path, "training.csv"), index=False)

#     # image_test = [os.path.join(data_path, 'sub-' + str(pth) + '_T1w.nii.gz') for pth in image_test]
#     validation_data = {"X": image_test, "Y": label_test}
#     df_validation = pd.DataFrame(validation_data)
#     df_validation.to_csv(os.path.join(out_path, "validation.csv"), index=False)

#     fold_no += 1

1035 1049
1035 1049


## Generate tfrecords for n-fold CV Dataset


In [8]:
import random
import nobrainer
import os, sys
sys.path.append("..")
import numpy as np
import nibabel as nb
from glob import glob
from pathlib import Path
from shutil import *
import subprocess
from operator import itemgetter
import pandas as pd

percentage = [100]

SPLITS = 15
for per in percentage:
    for fold in range(1, SPLITS+1):

        dir_path = "/work/06850/sbansal6/maverick2/mriqc-shared/experiment_faced_refaced/exp_face_refaced/exp_add_datasets/csv_F15_P{}/train_test_fold_{}/csv/".format(per, fold)
        tf_records_dir = "/work/06850/sbansal6/maverick2/mriqc-shared/experiment_faced_refaced/exp_face_refaced/exp_add_datasets/tfrecords_F15_P{}/tfrecords_fold_{}/".format(per, fold)
        os.makedirs(tf_records_dir, exist_ok=True)

        train_csv_path = os.path.join(dir_path, "training.csv")
        valid_csv_path = os.path.join(dir_path, "validation.csv")

        train_paths = pd.read_csv(train_csv_path)["X"].values
        train_labels = pd.read_csv(train_csv_path)["Y"].values
        train_D = list(zip(train_paths, train_labels))
        random.shuffle(train_D)
    #     print(train_D[0])

        valid_paths = pd.read_csv(valid_csv_path)["X"].values
        valid_labels = pd.read_csv(valid_csv_path)["Y"].values
        valid_D = list(zip(valid_paths, valid_labels))
        random.shuffle(valid_D)

        train_write_path = os.path.join(tf_records_dir, 'data-train_shard-{shard:03d}.tfrec')
        valid_write_path = os.path.join(tf_records_dir, 'data-valid_shard-{shard:03d}.tfrec')

        nobrainer.tfrecord.write(
            features_labels=train_D,
            filename_template=train_write_path,
            examples_per_shard=3)

        nobrainer.tfrecord.write(
            features_labels=valid_D,
            filename_template=valid_write_path,
            examples_per_shard=1)

138/138 [==============================] - 2s 16ms/step


## Generate Full Dataset

In [5]:
import os, sys
sys.path.append("..")
import numpy as np
import nibabel as nb
from glob import glob
from pathlib import Path
from shutil import *
import subprocess
from operator import itemgetter
from sklearn.model_selection import train_test_split
import pandas as pd
from random import shuffle

include = ['ds000140_anat',
           'ds000119_anat',
           'ds000157_anat',
           'ds002509_anat'
          ]

face_path = "/work/06850/sbansal6/maverick2/mriqc-shared/conformed/face"
# deface_path = "/work/06850/sbansal6/maverick2/mriqc-shared/conformed/deface"
face_defaced_path = "/work/06850/sbansal6/maverick2/mriqc-shared/conformed/face_defaced"
face_refaced_path = "/work/06850/sbansal6/maverick2/mriqc-shared/conformed/face_refaced"

paths_d = []
paths_f = []
labels_d = []
labels_f = []

for path in glob(face_defaced_path + "/*/*.nii*"):
    DS = path.split('/')[-2]
    paths_d.append(path)
    labels_d.append(0)
    
# for path in glob(deface_path + "/*/*.nii*"):
#     paths.append(path)
#     labels.append(0)

for path in glob(face_refaced_path + "/*/*.nii*"):
    DS = path.split('/')[-2]
    paths_f.append(path)
    labels_f.append(1)
    
# for path in glob(face_path + "/*/*.nii*"):
#     DS = path.split('/')[-2]
#     paths_f.append(path)
#     labels_f.append(1)
    
print(len(paths_f), len(paths_d))
print(len(labels_f), len(labels_d))

percentage = [25, 50, 75, 100]

for per in percentage:
#     print(len(paths))
    ind = int(len(paths_f)*per/100)
    shuffle(paths_d)
    shuffle(paths_f)
    temp_p = paths_d[:ind] + paths_f[:ind]
    temp_l = labels_d[:ind] + labels_f[:ind]
    save_path = "/work/06850/sbansal6/maverick2/mriqc-shared/experiment_faced_refaced/exp_face_refaced/exp_add_datasets/csv_{}".format(per)
    os.makedirs(save_path, exist_ok=True)
    
    train_data = {"X": temp_p , "Y": temp_l}
    df_train = pd.DataFrame(train_data)
    df_train.to_csv(os.path.join(save_path, "training.csv"), index=False)

# validation_data = {"X": X_test, "Y": y_test}
# df_validation = pd.DataFrame(validation_data)
# df_validation.to_csv(os.path.join(save_path, "validation.csv"), index=False)


1035 1049
1035 1049


In [6]:
import random
import nobrainer
import os, sys
sys.path.append("..")
import numpy as np
import nibabel as nb
from glob import glob
from pathlib import Path
from shutil import *
import subprocess
from operator import itemgetter
import pandas as pd

percentage = [25, 50, 75, 100]

for per in percentage:
    dir_path = "/work/06850/sbansal6/maverick2/mriqc-shared/experiment_faced_refaced/exp_face_refaced/exp_add_datasets/csv_{}".format(per)
    tf_records_dir = "/work/06850/sbansal6/maverick2/mriqc-shared/experiment_faced_refaced/exp_face_refaced/exp_add_datasets/tfrecords_{}".format(per)
    os.makedirs(tf_records_dir, exist_ok=True)
    
    train_csv_path = os.path.join(dir_path, "training.csv")
    # valid_csv_path = os.path.join(dir_path, "validation.csv")

    train_paths = pd.read_csv(train_csv_path)["X"].values
    train_labels = pd.read_csv(train_csv_path)["Y"].values
    train_D = list(zip(train_paths, train_labels))
    random.shuffle(train_D)

    # valid_paths = pd.read_csv(valid_csv_path)["X"].values
    # valid_labels = pd.read_csv(valid_csv_path)["Y"].values
    # valid_D = list(zip(valid_paths, valid_labels))
    # random.shuffle(valid_D)

    train_write_path = os.path.join(tf_records_dir, 'data-train_shard-{shard:03d}.tfrec')
    # valid_write_path = os.path.join(tf_records_dir, 'data-valid_shard-{shard:03d}.tfrec')

    nobrainer.tfrecord.write(
        features_labels=train_D,
        filename_template=train_write_path,
        examples_per_shard=3)

# nobrainer.tfrecord.write(
#     features_labels=valid_D,
#     filename_template=valid_write_path,
#     examples_per_shard=1)

690/690 [==============================] - 9s 13ms/step


In [ ]:
print(tf.random.uniform((1,)))

In [2]:
import nobrainer
from nobrainer.io import _is_gzipped
from nobrainer.volume import to_blocks
import sys, os
sys.path.append('../defacing')
from preprocessing.augmentation import VolumeAugmentations, SliceAugmentations
from helpers.utils import load_vol
import tensorflow_probability as tfp
import tensorflow as tf
import glob
import numpy as np

AUTOTUNE = tf.data.experimental.AUTOTUNE
ROOTDIR = '/work/06850/sbansal6/maverick2/mriqc-shared/experiment_faced_refaced/exp_face_defaced/tfrecords_F15'
DISTRIBUTION = load_vol('../defacing/helpers/distribution.nii.gz')[0]
DISTRIBUTION /= DISTRIBUTION.sum()
COM = np.unravel_index(int(np.sum(DISTRIBUTION.ravel()*np.arange(len(DISTRIBUTION.ravel())))/np.sum(DISTRIBUTION.ravel())), DISTRIBUTION.shape)


# sampling from augmented distribution is same as augmenting the sampled points
# augmenting distribution at every iteration is expensive, so this way
sampler = lambda n, distribution = DISTRIBUTION, threshold = 0.1: np.array([ np.unravel_index(
          np.random.choice(np.arange(np.prod(distribution.shape)),
                                     p = distribution.ravel()),
          distribution.shape) + (+1 if np.random.randn() > 0.5 else -1)*np.random.randint(0, 
                                        int(distribution.shape[0]*threshold) + 1, 3) for _ in range(n)]) 


three_d_augmentations = {'rotation': 0.5,
                         'translation': 0.5,
                         'noop': 0.3
                        }

augmentvolume = VolumeAugmentations(DISTRIBUTION, three_d_augmentations)

two_d_augmentations = {'rotation': 0.5,
                       'fliplr': 0.5,
                       'flipud': 0.5,
                       'zoom': 0.5,
                       'noop': 0.3
                      }

# augmentslice = VolumeAugmentations(DISTRIBUTION, two_d_augmentations)


def get_dataset(
    file_pattern,
    n_classes,
    batch_size,
    volume_shape,
    plane,
    n = 24,
    block_shape=None,
    n_epochs=None,
    mapping=None,
    augment=False,
    shuffle_buffer_size=None,
    num_parallel_calls=AUTOTUNE,
):

    """ Returns tf.data.Dataset after preprocessing from
    tfrecords for training and validation

    Parameters
    ----------
    file_pattern:

    n_classes:
    """

    files = glob.glob(file_pattern)

    if not files:
        raise ValueError("no files found for pattern '{}'".format(file_pattern))

    compressed = _is_gzipped(files[0])
    shuffle = bool(shuffle_buffer_size)

    ds = nobrainer.dataset.tfrecord_dataset(
        file_pattern=file_pattern,
        volume_shape=volume_shape,
        shuffle=shuffle,
        scalar_label=True,
        compressed=compressed,
        num_parallel_calls=num_parallel_calls,
    )

    # if augment:
    #     ds = ds.map(
    #         lambda x, y: tf.cond(
    #             tf.random.uniform((1,)) > 0.5,
    #             true_fn=lambda: apply_augmentations(x, y),
    #             false_fn=lambda: (x, y),
    #         ),
    #         num_parallel_calls=num_parallel_calls,
    #     )

    def _ss(x, y):
        if augment:
            if three_d_augmentations['noop'] < 1:
                x, y = augmentvolume(x,y)
        x, y = structural_slice(x, y, 
                                plane, 
                                n, 
                                augment, 
                                augmentvolume.distribution)
        return (x, y)

    ds = ds.map(_ss, num_parallel_calls)

    #     def _f(x, y):
    #         x = to_blocks(x, block_shape)
    #         n_blocks = x.shape[0]
    #         y = tf.repeat(y, n_blocks)
    #         return (x, y)
    #     ds = ds.map(_f, num_parallel_calls=num_parallel_calls)

    # This step is necessary because it reduces the extra dimension.
    # ds = ds.unbatch()


    ds = ds.prefetch(buffer_size=batch_size)

    if batch_size is not None:
        ds = ds.batch(batch_size=batch_size, drop_remainder=True)

    if shuffle_buffer_size:
        ds = ds.shuffle(buffer_size=shuffle_buffer_size)

    # Repeat the dataset n_epochs times
    ds = ds.repeat(n_epochs)

    return ds


def structural_slice(x, y, 
                plane, 
                n = 4, 
                augment = False, 
                distribution = DISTRIBUTION):

    """ Transpose dataset based on the plane

    Parameters
    ----------
    x:

    y:

    plane:
    
    n:

    augment:
    """

    threshold = 0.1 if augment else 0.0 
    options = ["axial", "coronal", "sagittal", "combined"]
    shape = np.array(x.shape)

    if isinstance(plane, str) and plane in options:
        idxs = sampler(n, 
                        distribution, 
                        threshold)

        if plane == "axial":
            idx = np.random.randint(shape[0]**0.5)
            midx = idxs[:, 0]
            x = x

        if plane == "coronal":
            idx = np.random.randint(shape[1]**0.5)
            midx = idxs[:, 1]
            x = tf.transpose(x, perm=[1, 2, 0])


        if plane == "sagittal":
            idx = np.random.randint(shape[2]**0.5)
            midx = idxs[:, 2]
            x = tf.transpose(x, perm=[2, 0, 1])


        if plane == "combined":
            temp = {}
            for op in options[:-1]:
                temp[op] = structural_slice(x, y, 
                                            op, 
                                            n, 
                                            augment, 
                                            distribution)[0]
            x = temp

        if not plane == "combined":
            x = tf.squeeze(tf.gather_nd(x, midx.reshape(n, 1, 1)), axis=1)
            x = tf.math.reduce_mean(x, axis=0)
            x = tf.expand_dims(x, axis=-1)
            
            if augment:
                x = two_d_augmentations(x)
                
            x = tf.convert_to_tensor(x)
        return x, y
    else:
        raise ValueError("expected plane to be one of ['axial', 'coronal', 'sagittal']")




if __name__ == "__main__":

    n_classes = 2
    global_batch_size = 8
    volume_shape = (64, 64, 64)
    ds = get_dataset(
        os.path.join(ROOTDIR, "tfrecords_fold_1/data-train_*"),
        n_classes=n_classes,
        batch_size=global_batch_size,
        volume_shape=volume_shape,
        plane="sagittal",
        augment = False,
        shuffle_buffer_size=3,
    )

    import matplotlib
    matplotlib.use('Agg')
    import matplotlib.pyplot as plt

    print(ds)
#     for ii in range(100):
#         x,y=next(ds.as_numpy_iterator())
#         # print (np.min(x), np.max(x), np.unique(y))
#         count = 1
#         for i in range(global_batch_size):
#             for key in x.keys():
#                 plt.subplot(global_batch_size, 3, count)
#                 plt.imshow(x[key][i, :, :, 0])
#                 plt.title(str(y[i]))
#                 plt.xticks([]," ")
#                 plt.yticks([], " ")
#                 count += 1
#         plt.savefig("processed_image_combined_{}.png".format(ii))


# dataset_train_coronal = get_dataset("tfrecords/tfrecords_fold_1/data-train_*",
#                             n_classes=n_classes,
#                             batch_size=global_batch_size,
#                             volume_shape=volume_shape,
#                             block_shape=block_shape,
#                             plane='coronal',
#                             shuffle_buffer_size=3)

# dataset_train_sagittal = get_dataset("tfrecords/tfrecords_fold_1/data-train_*",
#                             n_classes=n_classes,
#                             batch_size=global_batch_size,
#                             volume_shape=volume_shape,
#                             block_shape=block_shape,
#                             plane='sagittal',
#                             shuffle_buffer_size=3)


<RepeatDataset shapes: ((8, 64, 64, 1), (8, 1)), types: (tf.float32, tf.float32)>


In [15]:
import pandas as pd
fold = 1
ROOTDIR = '/work/06850/sbansal6/maverick2/mriqc-shared/experiment_faced_refaced/exp_face_defaced/tfrecords_full'
dir_path = '/work/06850/sbansal6/maverick2/mriqc-shared/experiment_faced_refaced/exp_face_defaced/csv_full'

train_csv_path = os.path.join(dir_path, "training.csv")
# valid_csv_path = os.path.join(dir_path, "validation.csv")

train_paths = pd.read_csv(train_csv_path)["X"].values
# valid_paths = pd.read_csv(valid_csv_path)["X"].values

train_labels = pd.read_csv(train_csv_path)["Y"].values
# valid_labels = pd.read_csv(valid_csv_path)["Y"].values

print(type(train_labels))

from sklearn.utils import class_weight

weights = class_weight.compute_class_weight('balanced',
                                            np.unique(train_labels),
                                            train_labels)

print(dict(enumerate(weights)))

<class 'numpy.ndarray'>
{0: 1.0, 1: 1.0}


In [9]:
# Std packages
import sys, os
import glob
import math

sys.path.append("../defacing")

# Custom packages
from models import modelN
# from dataloaders.dataset import get_dataset

# Tf packages
import tensorflow as tf
import pandas as pd
import numpy as np
from sklearn.utils import class_weight
from tensorflow.keras import backend as K
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import (
    ModelCheckpoint,
    LearningRateScheduler,
    TensorBoard,
)
# import nobrainer
from tensorflow.keras import metrics
from tensorflow.keras import losses

ROOTDIR = '/work/06850/sbansal6/maverick2/mriqc-shared/experiment_faced_refaced/exp_face_refaced/exp_add_datasets'


def scheduler(epoch):
    if epoch < 3:
        return 0.001
    else:
        return 0.001 * tf.math.exp(0.1 * (10 - epoch))


def train(
    volume_shape=(64, 64, 64),
    image_size=(64, 64),
    dropout=0.2,
    batch_size=16,
    n_classes=2,
    n_epochs=15,
    fold=1,
    percent=100,
):
    
    
    print("FOLD {}".format(fold))
    dir_path = os.path.join(ROOTDIR, "csv_F15_P{}/train_test_fold_{}/csv".format(percent, fold))
#     dir_path = os.path.join(ROOTDIR, "csv_{}".format(percent))

    train_csv_path = os.path.join(dir_path, "training.csv")
    valid_csv_path = os.path.join(dir_path, "validation.csv")

    train_paths = pd.read_csv(train_csv_path)["X"].values
    valid_paths = pd.read_csv(valid_csv_path)["X"].values
    
    train_labels = pd.read_csv(train_csv_path)["Y"].values
    valid_labels = pd.read_csv(valid_csv_path)["Y"].values
    
    weights = class_weight.compute_class_weight('balanced',
                                                np.unique(train_labels),
                                                train_labels)
    weights = dict(enumerate(weights))
    
    print(weights)
    
    planes = ["axial", "coronal", "sagittal", "combined"]

#     strategy = tf.distribute.MirroredStrategy()
#     BATCH_SIZE_PER_REPLICA = batch_size
#     global_batch_size = BATCH_SIZE_PER_REPLICA * strategy.num_replicas_in_sync

    global_batch_size = batch_size
    
    model_save_path = os.path.join(ROOTDIR, "model_save_dir_F15_P{}/train_test_fold_{}".format(percent, fold))
#     model_save_path = os.path.join(ROOTDIR, "model_save_dir_{}".format(percent))

    
    os.makedirs(model_save_path, exist_ok=True)

    cp_save_path = os.path.join(model_save_path, "weights")

    logdir_path = os.path.join(model_save_path, "tb_logs")
    if not os.path.exists(logdir_path):
        os.makedirs(logdir_path)

    for plane in planes:

        logdir = os.path.join(logdir_path, plane)
        os.makedirs(logdir, exist_ok=True)

        tbCallback = TensorBoard(
            log_dir=logdir, histogram_freq=1, write_graph=True, write_images=True,
        )

        os.makedirs(os.path.join(cp_save_path, plane), exist_ok=True)

        model_checkpoint = ModelCheckpoint(
            os.path.join(cp_save_path, plane, "best-wts.h5"),
            monitor="val_loss",
            save_weights_only=True,
            mode="min",
        )

#         with strategy.scope():

        if not plane == "combined": 
            lr = 1e-3
            model = modelN.Submodel(
                input_shape=image_size,
                dropout=dropout,
                name=plane,
                include_top=True,
                weights=None,
            )
        else:
            lr = 5e-4
            model = modelN.CombinedClassifier(
                input_shape=image_size,
                dropout=dropout,
                trainable=True,
                wts_root=cp_save_path,
            )

        print("Submodel: ", plane)
#         print(model.summary())

        METRICS = [
            metrics.TruePositives(name="tp"),
            metrics.FalsePositives(name="fp"),
            metrics.TrueNegatives(name="tn"),
            metrics.FalseNegatives(name="fn"),
            metrics.BinaryAccuracy(name="accuracy"),
            metrics.Precision(name="precision"),
            metrics.Recall(name="recall"),
            metrics.AUC(name="auc"),
        ]

        model.compile(
            loss=tf.keras.losses.binary_crossentropy,
            optimizer=Adam(learning_rate=lr),
            metrics=METRICS,
        )

        print("GLOBAL BATCH SIZE: ", global_batch_size)

        dataset_train = get_dataset(
            file_pattern=os.path.join(ROOTDIR, "tfrecords_F15_P{}/tfrecords_fold_{}/data-train_*".format(percent, fold)),
            n_classes=n_classes,
            batch_size=global_batch_size,
            volume_shape=volume_shape,
            plane=plane,
            shuffle_buffer_size=global_batch_size,
        )

        dataset_valid = get_dataset(
            file_pattern=os.path.join(ROOTDIR, "tfrecords_F15_P{}/tfrecords_fold_{}/data-valid_*".format(percent, fold)),
            n_classes=n_classes,
            batch_size=global_batch_size,
            volume_shape=volume_shape,
            plane=plane,
            shuffle_buffer_size=global_batch_size,
        )
        
        steps_per_epoch = math.ceil(len(train_paths)/batch_size)
        validation_steps = math.ceil(len(valid_paths)/batch_size)

        print(steps_per_epoch)
    

        lrcallback = tf.keras.callbacks.LearningRateScheduler(scheduler)
        
        model.fit(
            dataset_train,
            epochs=n_epochs,
            steps_per_epoch=steps_per_epoch,
            validation_data=dataset_valid,
            validation_steps=validation_steps,
            callbacks=[tbCallback, model_checkpoint],
            class_weight = weights,
        )

        del model
        K.clear_session()


if __name__ == "__main__":
    percentage = [50, 75]
#     for per in percentage:
    for fold in range(1, 15):
        train(fold=fold)


FOLD 1
{0: 1.0, 1: 1.0}
Submodel:  axial
GLOBAL BATCH SIZE:  16
121
Train for 121 steps, validate for 9 steps
Epoch 1/15
121/121 [==============================] - 6s 46ms/step - loss: 0.1620 - tp: 923.0000 - fp: 44.0000 - tn: 923.0000 - fn: 46.0000 - accuracy: 0.9535 - precision: 0.9545 - recall: 0.9525 - auc: 0.9869 - val_loss: 0.7677 - val_tp: 71.0000 - val_fp: 73.0000 - val_tn: 0.0000e+00 - val_fn: 0.0000e+00 - val_accuracy: 0.4931 - val_precision: 0.4931 - val_recall: 1.0000 - val_auc: 0.9910
Epoch 2/15
121/121 [==============================] - 2s 20ms/step - loss: 0.0132 - tp: 965.0000 - fp: 5.0000 - tn: 960.0000 - fn: 6.0000 - accuracy: 0.9943 - precision: 0.9948 - recall: 0.9938 - auc: 0.9999 - val_loss: 1.0531 - val_tp: 69.0000 - val_fp: 75.0000 - val_tn: 0.0000e+00 - val_fn: 0.0000e+00 - val_accuracy: 0.4792 - val_precision: 0.4792 - val_recall: 1.0000 - val_auc: 0.9947
Epoch 3/15
121/121 [==============================] - 3s 21ms/step - loss: 0.0068 - tp: 957.0000 - fp: 2.0

Epoch 12/15
121/121 [==============================] - 3s 22ms/step - loss: 9.0421e-05 - tp: 970.0000 - fp: 0.0000e+00 - tn: 966.0000 - fn: 0.0000e+00 - accuracy: 1.0000 - precision: 1.0000 - recall: 1.0000 - auc: 1.0000 - val_loss: 0.0202 - val_tp: 72.0000 - val_fp: 2.0000 - val_tn: 70.0000 - val_fn: 0.0000e+00 - val_accuracy: 0.9861 - val_precision: 0.9730 - val_recall: 1.0000 - val_auc: 1.0000
Epoch 13/15
121/121 [==============================] - 3s 24ms/step - loss: 4.3507e-05 - tp: 980.0000 - fp: 0.0000e+00 - tn: 956.0000 - fn: 0.0000e+00 - accuracy: 1.0000 - precision: 1.0000 - recall: 1.0000 - auc: 1.0000 - val_loss: 0.0168 - val_tp: 71.0000 - val_fp: 1.0000 - val_tn: 72.0000 - val_fn: 0.0000e+00 - val_accuracy: 0.9931 - val_precision: 0.9861 - val_recall: 1.0000 - val_auc: 1.0000
Epoch 14/15
121/121 [==============================] - 3s 22ms/step - loss: 3.1194e-05 - tp: 955.0000 - fp: 0.0000e+00 - tn: 981.0000 - fn: 0.0000e+00 - accuracy: 1.0000 - precision: 1.0000 - recall: 

GLOBAL BATCH SIZE:  16
121
Train for 121 steps, validate for 9 steps
Epoch 1/15
121/121 [==============================] - 8s 65ms/step - loss: 0.3481 - tp: 901.0000 - fp: 67.0000 - tn: 900.0000 - fn: 68.0000 - accuracy: 0.9303 - precision: 0.9308 - recall: 0.9298 - auc: 0.9671 - val_loss: 0.6479 - val_tp: 0.0000e+00 - val_fp: 0.0000e+00 - val_tn: 72.0000 - val_fn: 72.0000 - val_accuracy: 0.5000 - val_precision: 0.0000e+00 - val_recall: 0.0000e+00 - val_auc: 0.9983
Epoch 2/15
121/121 [==============================] - 4s 29ms/step - loss: 0.0209 - tp: 957.0000 - fp: 4.0000 - tn: 967.0000 - fn: 8.0000 - accuracy: 0.9938 - precision: 0.9958 - recall: 0.9917 - auc: 0.9996 - val_loss: 0.3157 - val_tp: 72.0000 - val_fp: 22.0000 - val_tn: 50.0000 - val_fn: 0.0000e+00 - val_accuracy: 0.8472 - val_precision: 0.7660 - val_recall: 1.0000 - val_auc: 1.0000
Epoch 3/15
121/121 [==============================] - 4s 29ms/step - loss: 0.0136 - tp: 960.0000 - fp: 3.0000 - tn: 968.0000 - fn: 5.0000 - ac

121/121 [==============================] - 2s 19ms/step - loss: 4.9911e-04 - tp: 971.0000 - fp: 0.0000e+00 - tn: 965.0000 - fn: 0.0000e+00 - accuracy: 1.0000 - precision: 1.0000 - recall: 1.0000 - auc: 1.0000 - val_loss: 1.1053e-04 - val_tp: 70.0000 - val_fp: 0.0000e+00 - val_tn: 74.0000 - val_fn: 0.0000e+00 - val_accuracy: 1.0000 - val_precision: 1.0000 - val_recall: 1.0000 - val_auc: 1.0000
Epoch 7/15
121/121 [==============================] - 2s 19ms/step - loss: 2.3074e-04 - tp: 968.0000 - fp: 0.0000e+00 - tn: 968.0000 - fn: 0.0000e+00 - accuracy: 1.0000 - precision: 1.0000 - recall: 1.0000 - auc: 1.0000 - val_loss: 1.0678e-04 - val_tp: 74.0000 - val_fp: 0.0000e+00 - val_tn: 70.0000 - val_fn: 0.0000e+00 - val_accuracy: 1.0000 - val_precision: 1.0000 - val_recall: 1.0000 - val_auc: 1.0000
Epoch 8/15
121/121 [==============================] - 2s 19ms/step - loss: 1.9936e-04 - tp: 971.0000 - fp: 0.0000e+00 - tn: 965.0000 - fn: 0.0000e+00 - accuracy: 1.0000 - precision: 1.0000 - recall

GLOBAL BATCH SIZE:  16
121
Train for 121 steps, validate for 9 steps
Epoch 1/15
121/121 [==============================] - 8s 69ms/step - loss: 0.3166 - tp: 905.0000 - fp: 57.0000 - tn: 913.0000 - fn: 61.0000 - accuracy: 0.9390 - precision: 0.9407 - recall: 0.9369 - auc: 0.9725 - val_loss: 0.7097 - val_tp: 72.0000 - val_fp: 72.0000 - val_tn: 0.0000e+00 - val_fn: 0.0000e+00 - val_accuracy: 0.5000 - val_precision: 0.5000 - val_recall: 1.0000 - val_auc: 0.9841
Epoch 2/15
121/121 [==============================] - 4s 37ms/step - loss: 0.0201 - tp: 959.0000 - fp: 7.0000 - tn: 963.0000 - fn: 7.0000 - accuracy: 0.9928 - precision: 0.9928 - recall: 0.9928 - auc: 0.9993 - val_loss: 0.2409 - val_tp: 71.0000 - val_fp: 15.0000 - val_tn: 58.0000 - val_fn: 0.0000e+00 - val_accuracy: 0.8958 - val_precision: 0.8256 - val_recall: 1.0000 - val_auc: 1.0000
Epoch 3/15
121/121 [==============================] - 4s 30ms/step - loss: 0.0284 - tp: 967.0000 - fp: 9.0000 - tn: 953.0000 - fn: 7.0000 - accuracy: 

...

GLOBAL BATCH SIZE:  16
121
Train for 121 steps, validate for 9 steps
Epoch 1/15
121/121 [==============================] - 8s 65ms/step - loss: 0.3767 - tp: 908.0000 - fp: 59.0000 - tn: 907.0000 - fn: 62.0000 - accuracy: 0.9375 - precision: 0.9390 - recall: 0.9361 - auc: 0.9646 - val_loss: 0.6245 - val_tp: 73.0000 - val_fp: 49.0000 - val_tn: 19.0000 - val_fn: 3.0000 - val_accuracy: 0.6389 - val_precision: 0.5984 - val_recall: 0.9605 - val_auc: 0.8156
Epoch 2/15
121/121 [==============================] - 4s 30ms/step - loss: 0.0126 - tp: 959.0000 - fp: 2.0000 - tn: 969.0000 - fn: 6.0000 - accuracy: 0.9959 - precision: 0.9979 - recall: 0.9938 - auc: 0.9999 - val_loss: 0.4676 - val_tp: 76.0000 - val_fp: 40.0000 - val_tn: 28.0000 - val_fn: 0.0000e+00 - val_accuracy: 0.7222 - val_precision: 0.6552 - val_recall: 1.0000 - val_auc: 0.9819
Epoch 3/15
121/121 [==============================] - 4s 31ms/step - loss: 0.0072 - tp: 971.0000 - fp: 3.0000 - tn: 959.0000 - fn: 3.0000 - accuracy: 0.9969 

Epoch 12/15
121/121 [==============================] - 3s 22ms/step - loss: 0.0013 - tp: 965.0000 - fp: 1.0000 - tn: 970.0000 - fn: 0.0000e+00 - accuracy: 0.9995 - precision: 0.9990 - recall: 1.0000 - auc: 1.0000 - val_loss: 0.3751 - val_tp: 69.0000 - val_fp: 2.0000 - val_tn: 72.0000 - val_fn: 1.0000 - val_accuracy: 0.9792 - val_precision: 0.9718 - val_recall: 0.9857 - val_auc: 0.9793
Epoch 13/15
121/121 [==============================] - 3s 22ms/step - loss: 0.0075 - tp: 972.0000 - fp: 2.0000 - tn: 960.0000 - fn: 2.0000 - accuracy: 0.9979 - precision: 0.9979 - recall: 0.9979 - auc: 1.0000 - val_loss: 0.1189 - val_tp: 69.0000 - val_fp: 1.0000 - val_tn: 74.0000 - val_fn: 0.0000e+00 - val_accuracy: 0.9931 - val_precision: 0.9857 - val_recall: 1.0000 - val_auc: 0.9930
Epoch 14/15
121/121 [==============================] - 3s 22ms/step - loss: 0.0143 - tp: 945.0000 - fp: 3.0000 - tn: 984.0000 - fn: 4.0000 - accuracy: 0.9964 - precision: 0.9968 - recall: 0.9958 - auc: 0.9994 - val_loss: 0.0

GLOBAL BATCH SIZE:  16
121
Train for 121 steps, validate for 9 steps
Epoch 1/15
121/121 [==============================] - 8s 65ms/step - loss: 0.3631 - tp: 897.0000 - fp: 73.0000 - tn: 888.0000 - fn: 78.0000 - accuracy: 0.9220 - precision: 0.9247 - recall: 0.9200 - auc: 0.9608 - val_loss: 0.6580 - val_tp: 73.0000 - val_fp: 71.0000 - val_tn: 0.0000e+00 - val_fn: 0.0000e+00 - val_accuracy: 0.5069 - val_precision: 0.5069 - val_recall: 1.0000 - val_auc: 0.9771
Epoch 2/15
121/121 [==============================] - 4s 29ms/step - loss: 0.0230 - tp: 948.0000 - fp: 4.0000 - tn: 973.0000 - fn: 11.0000 - accuracy: 0.9923 - precision: 0.9958 - recall: 0.9885 - auc: 0.9992 - val_loss: 0.4199 - val_tp: 71.0000 - val_fp: 51.0000 - val_tn: 22.0000 - val_fn: 0.0000e+00 - val_accuracy: 0.6458 - val_precision: 0.5820 - val_recall: 1.0000 - val_auc: 0.9977
Epoch 3/15
121/121 [==============================] - 4s 30ms/step - loss: 0.0091 - tp: 969.0000 - fp: 3.0000 - tn: 960.0000 - fn: 4.0000 - accuracy:

121/121 [==============================] - 2s 21ms/step - loss: 3.2867e-05 - tp: 960.0000 - fp: 0.0000e+00 - tn: 976.0000 - fn: 0.0000e+00 - accuracy: 1.0000 - precision: 1.0000 - recall: 1.0000 - auc: 1.0000 - val_loss: 0.1153 - val_tp: 75.0000 - val_fp: 2.0000 - val_tn: 67.0000 - val_fn: 0.0000e+00 - val_accuracy: 0.9861 - val_precision: 0.9740 - val_recall: 1.0000 - val_auc: 0.9855
Epoch 12/15
121/121 [==============================] - 3s 21ms/step - loss: 3.1852e-05 - tp: 962.0000 - fp: 0.0000e+00 - tn: 974.0000 - fn: 0.0000e+00 - accuracy: 1.0000 - precision: 1.0000 - recall: 1.0000 - auc: 1.0000 - val_loss: 0.0564 - val_tp: 73.0000 - val_fp: 1.0000 - val_tn: 70.0000 - val_fn: 0.0000e+00 - val_accuracy: 0.9931 - val_precision: 0.9865 - val_recall: 1.0000 - val_auc: 0.9930
Epoch 13/15
121/121 [==============================] - 3s 22ms/step - loss: 1.7585e-04 - tp: 966.0000 - fp: 0.0000e+00 - tn: 970.0000 - fn: 0.0000e+00 - accuracy: 1.0000 - precision: 1.0000 - recall: 1.0000 - auc

GLOBAL BATCH SIZE:  16
121
Train for 121 steps, validate for 9 steps
Epoch 1/15
121/121 [==============================] - 8s 67ms/step - loss: 0.1697 - tp: 935.0000 - fp: 34.0000 - tn: 934.0000 - fn: 33.0000 - accuracy: 0.9654 - precision: 0.9649 - recall: 0.9659 - auc: 0.9851 - val_loss: 0.7204 - val_tp: 73.0000 - val_fp: 71.0000 - val_tn: 0.0000e+00 - val_fn: 0.0000e+00 - val_accuracy: 0.5069 - val_precision: 0.5069 - val_recall: 1.0000 - val_auc: 0.9830
Epoch 2/15
121/121 [==============================] - 4s 30ms/step - loss: 0.0153 - tp: 959.0000 - fp: 6.0000 - tn: 965.0000 - fn: 6.0000 - accuracy: 0.9938 - precision: 0.9938 - recall: 0.9938 - auc: 0.9999 - val_loss: 1.0184 - val_tp: 71.0000 - val_fp: 73.0000 - val_tn: 0.0000e+00 - val_fn: 0.0000e+00 - val_accuracy: 0.4931 - val_precision: 0.4931 - val_recall: 1.0000 - val_auc: 0.9997
Epoch 3/15
121/121 [==============================] - 4s 30ms/step - loss: 0.0053 - tp: 963.0000 - fp: 2.0000 - tn: 969.0000 - fn: 2.0000 - accurac

## Inference

In [3]:
import os
import sys
import time
import imgaug
import shutil
import numpy as np
from imgaug import augmenters as iaa
import matplotlib.pyplot as plt

sys.path.append('../defacing')
from helpers.utils import save_vol, load_vol
from preprocessing.conform import conform_data
from preprocessing.normalization import clip, standardize, normalize

import tensorflow as tf


class DataGeneratoronFly(object):
    """
    """

    def __init__(self, conform_size=(64, 64, 64),
                        conform_zoom=(4., 4., 4.), 
                        nchannels=1, 
                        nruns=8,
                        nsamples=20,
                        save=False, 
                        transform=None):

        self.conform_size=conform_size
        self.conform_zoom=conform_zoom
        self.nchannels=nchannels
        self.transform=transform
        self.nsamples=nsamples
        self.nruns=nruns
        self.save=save
        DISTRIBUTION = load_vol('../defacing/helpers/distribution.nii.gz')[0]
        assert DISTRIBUTION.shape == conform_size, "Invalid conform_size needs to regenerate face distribution"

        DISTRIBUTION /= DISTRIBUTION.sum()
        self.sampler = lambda n: np.array([ np.unravel_index(
                  np.random.choice(np.arange(np.prod(DISTRIBUTION.shape)),
                                             p = DISTRIBUTION.ravel()),
                  DISTRIBUTION.shape) for _ in range(n)]) 



    def _augmentation(self, volume):
        r"""
                Augmenters that are safe to apply to masks
                Some, such as Affine, have settings that make them unsafe, so always
                test your augmentation on masks
        """
        volume_shape = volume.shape
        det = self.transform.to_deterministic()
        volume = det.augment_image(volume)

        assert volume.shape == volume_shape, "Augmentation shouldn't change volume size"
        return volume


    def _sample_slices(self, volume, plane=None):

        options = ["axial", "coronal", "sagittal", "combined"]
        assert plane in options, "expected plane to be one of ['axial', 'coronal', 'sagittal']"
        samples = self.sampler(self.nsamples)

        if plane == "sagittal":
            midx = samples[:, 0]
            volume = volume
            k = 3

        if plane == "coronal":
            midx = samples[:, 1]
            volume = np.transpose(volume, axes=[1, 2, 0])
            k = 2

        if plane == "axial":
            midx = samples[:, 2]
            volume = np.transpose(volume, axes=[2, 0, 1])
            k = 1

        if plane == "combined":
            temp = []
            for op in options[:-1]:
                temp.append(self._sample_slices(volume, op))
            volume = temp
#             plt.subplot(1, 3, 1)
#             plt.imshow(volume[0][:,:,0])
#             plt.subplot(1, 3, 2)
#             plt.imshow(volume[1][:,:,0])
#             plt.subplot(1, 3, 3)
#             plt.imshow(volume[2][:,:,0])
#             plt.show()

        if not plane == "combined":
            volume = np.squeeze(volume[midx,:,:])
            volume = np.mean(volume, axis=0)
            volume = np.rot90(volume, k)
            volume = volume[..., None]
        return volume


    def get_data(self, volume):
        # Generate indexes of the batch
#         volume = clip(volume, q=90)
#         volume = normalize(volume)
#         volume = standardize(volume)
#         newaffine = np.eye(4)
#         newaffine[:3, 3] = -0.5 * (np.array(self.conform_size) - 1)
#         os.makedirs('./tmp', exist_ok=True)
#         save_vol('./tmp/Pre-processing.nii.gz', volume, newaffine)
#         conform_data('./tmp/Pre-processing.nii.gz',
#                         './tmp/Conformed.nii.gz',
#                         self.conform_size,
#                         self.conform_zoom)

#         volume = load_vol('./tmp/Conformed.nii.gz')[0]

        if self.transform:
            volume = self._augmentation(volume)

        slices = []
        for _ in range(self.nruns):
            slices.append(self._sample_slices(volume, 
                                    plane="combined"))

#         if not self.save: 
#             shutil.rmtree('./tmp')
        return slices

    
if __name__=="__main__":
    
    path = '../sample_vols/defaced/example2.nii.gz'
    
    vol, _, _ = load_vol(path)
    
    
#     inference_transform_params = {
#             "conform_size": (64, 64, 64),
#             "conform_zoom": (4., 4., 4.), 
#             "nchannels": 1, 
#             "nruns": 8,
#             "nsamples": 24,
#             "save": False, 
#             "transform": None
#         }
    
#     inference_generator = DataGeneratoronFly(**inference_transform_params)
    
#     slices = inference_generator.get_data(vol)
    
#     slices = np.transpose(np.array(slices),axes=[1, 0, 2, 3, 4])
#     ds = {}
#     ds['axial'] = slices[0]
#     ds['coronal'] = slices[1]
#     ds['sagittal'] = slices[2]
    
# #     print(ds)
#     print(slices.shape)
#         return slices

In [5]:
import os
import sys
import numpy as np
import tensorflow as tf
import timeit

sys.path.append('../defacing')
from tensorflow.keras import backend as K
from tensorflow.keras.models import load_model
from models.modelN import CombinedClassifier
# from dataloaders.inference_dataloader import DataGeneratoronFly

ROOTDIR = "/work/06850/sbansal6/maverick2/mriqc-shared/experiment_faced_refaced/exp_face_defaced/exp_add_datasets"


class inferer(object):
    """
       nMontecarlo: for multiple exp for same model
       quick: checks for all 3 fold models
       mode: method to merge predictions
             allowed ['avg', 'max_vote']
    """

    def __init__(self, percent=25, nMontecarlo=8, mode="avg"):
        r"""
        """
        inference_transform_params = {
            "conform_size": (64, 64, 64),
            "conform_zoom": (4., 4., 4.), 
            "nchannels": 1, 
            "nruns": 8,
            "nsamples": 32,
            "save": False, 
            "transform": None
        }

        self.mode = mode
        assert self.mode.lower() in [
            "avg",
            "max_vote",
        ], "unknown mode, allowed mode are ['avg', 'max_vote']"

        self.inference_generator = DataGeneratoronFly(**inference_transform_params)
        self.model = CombinedClassifier(
            input_shape=(64, 64), dropout=0.4, wts_root=None, trainable=True
        )
        self.model.load_weights(

            os.path.abspath(os.path.join(ROOTDIR, "model_save_dir_{}/weights/combined/best-wts.h5".format(percent)))
        )

    def infer(self, vol):
        """
        vol : can be numpy ndarray or path to volume
        """
        slices = self.inference_generator.get_data(vol)
        slices = np.transpose(np.array(slices),axes=[1, 0, 2, 3, 4])
        
        ds = {}
        ds['axial'] = slices[0]
        ds['coronal'] = slices[1]
        ds['sagittal'] = slices[2]
    
        predictions = self.model.predict(ds)

        if self.mode.lower() == "max_vote":
            predictions = np.round(predictions)
            unique_elements = np.unique(predictions)
            count_array = np.array(
                [
                    sum(predictions == unique_element)
                    for unique_element in unique_elements
                ]
            )
            pred = (
                np.argmax(count_array) if len(count_array) > 1 else unique_elements[0]
            )
            conf = (
                1
                if len(count_array) == 1
                else count_array[pred] * 1.0 / np.sum(count_array)
            )
        elif self.mode.lower() == "avg":
            conf = np.mean(predictions)
            pred = np.round(conf)

        pred_str = "faced" if pred == 1 else "defaced"
        conf = conf if pred == 1 else 1.0 - conf

        print("[INFO] Given volume is " + pred_str + " with confidence of: {}".format(conf))
        
#         del self.model
#         K.clear_session()
        
        return pred, conf


In [3]:
import sys
import glob
sys.path.append('../defacing')
from helpers.utils import load_vol
import pandas as pd
# from defacing.inference import inferer

percentage = [25, 50, 75, 100]
for per in percentage:
    _inferer = inferer(percent=per)
    # path = '../sample_vols/faced/example1.nii.gz'
    paths = glob.glob('/work/01329/poldrack/data/mriqc-net/data/test_images/test1_images/*/*')
#     paths = glob.glob('/work/01329/poldrack/data/mriqc-net/data/defaced/ds002236_anat/*')

    # print(len(paths))
    inf = {}

    for file in paths:
        print(file)
        temp = {}
        vol, _, _ = load_vol(file)
        label, conf = _inferer.infer(vol)
        temp['label'] = label
        temp['confidence'] = conf
        inf[file.split('/')[-2] + '/' + file.split('/')[-1]] = temp
    df = pd.DataFrame.from_dict(inf).transpose()
    df.to_csv('test_inference_dstest_model_fdefaced_{}.csv'.format(per))


Model: "model_3"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
axial (InputLayer)              [(None, 64, 64, 1)]  0                                            
__________________________________________________________________________________________________
sagittal (InputLayer)           [(None, 64, 64, 1)]  0                                            
__________________________________________________________________________________________________
coronal (InputLayer)            [(None, 64, 64, 1)]  0                                            
__________________________________________________________________________________________________
conv2d (Conv2D)                 (None, 64, 64, 8)    80          axial[0][0]                      
____________________________________________________________________________________________

[-34. -34.  -2.]
[INFO] Given volume is faced with confidence of: 0.907974898815155
/work/01329/poldrack/data/mriqc-net/data/test_images/test1_images/ds002330/sub-07_T1w.nii.gz
[-34. -34.  -2.]
[INFO] Given volume is faced with confidence of: 0.9572145342826843
/work/01329/poldrack/data/mriqc-net/data/test_images/test1_images/ds002330/sub-65_T1w.nii.gz
[-34. -34.  -2.]
[INFO] Given volume is faced with confidence of: 0.7246092557907104
/work/01329/poldrack/data/mriqc-net/data/test_images/test1_images/ds002330/sub-19_T1w.nii.gz
[-34. -34.  -2.]
[INFO] Given volume is faced with confidence of: 0.9539231061935425
/work/01329/poldrack/data/mriqc-net/data/test_images/test1_images/ds002330/sub-48_T1w.nii.gz
[-34. -34.  -2.]
[INFO] Given volume is faced with confidence of: 0.8872464299201965
/work/01329/poldrack/data/mriqc-net/data/test_images/test1_images/ds002330/sub-08_T1w.nii.gz
[-34. -34.  -2.]
[INFO] Given volume is faced with confidence of: 0.938897967338562
/work/01329/poldrack/data/m

[INFO] Given volume is faced with confidence of: 0.8688989877700806
/work/01329/poldrack/data/mriqc-net/data/test_images/test1_images/ds002330/sub-59_T1w.nii.gz
[-34. -34.  -2.]
[INFO] Given volume is faced with confidence of: 0.8203302621841431
/work/01329/poldrack/data/mriqc-net/data/test_images/test1_images/ds002330/sub-41_T1w.nii.gz
[-34. -34.  -2.]
[INFO] Given volume is faced with confidence of: 0.8828160166740417
/work/01329/poldrack/data/mriqc-net/data/test_images/test1_images/ds002330/sub-35_T1w.nii.gz
[-34. -34.  -2.]
[INFO] Given volume is faced with confidence of: 0.947839617729187
/work/01329/poldrack/data/mriqc-net/data/test_images/test1_images/ds002330/sub-25_T1w.nii.gz
[-34. -34.  -2.]
[INFO] Given volume is faced with confidence of: 0.9424105882644653
/work/01329/poldrack/data/mriqc-net/data/test_images/test1_images/ds002330/sub-58_T1w.nii.gz
[-34. -34.  -2.]
[INFO] Given volume is faced with confidence of: 0.8813940286636353
/work/01329/poldrack/data/mriqc-net/data/te

[54. -2. -2.]
[INFO] Given volume is faced with confidence of: 0.9960217475891113
/work/01329/poldrack/data/mriqc-net/data/test_images/test1_images/ds000148/sub-33_acq-sagittal_T1w.nii.gz
[50. -2. -2.]
[INFO] Given volume is faced with confidence of: 0.9989414215087891
/work/01329/poldrack/data/mriqc-net/data/test_images/test1_images/ds000148/sub-39_acq-sagittal_T1w.nii.gz
[50. -2. -2.]
[INFO] Given volume is faced with confidence of: 0.9926496744155884
/work/01329/poldrack/data/mriqc-net/data/test_images/test1_images/ds000148/sub-16_acq-sagittal_T1w.nii.gz
[50. -2. -2.]
[INFO] Given volume is faced with confidence of: 0.9942286014556885
/work/01329/poldrack/data/mriqc-net/data/test_images/test1_images/ds000148/sub-27_acq-sagittal_T1w.nii.gz
[50. -2. -2.]
[INFO] Given volume is faced with confidence of: 0.9840285778045654
/work/01329/poldrack/data/mriqc-net/data/test_images/test1_images/ds000148/sub-06_acq-sagittal_T1w.nii.gz
[50. -2. -2.]
[INFO] Given volume is faced with confidence o

[ 30.  -4. -34.]
[INFO] Given volume is faced with confidence of: 0.9770197868347168
/work/01329/poldrack/data/mriqc-net/data/test_images/test1_images/ds001919/sub-oxfordFmrib06_T1w.nii.gz
[ 30.  -4. -34.]
[INFO] Given volume is faced with confidence of: 0.9852232933044434
/work/01329/poldrack/data/mriqc-net/data/test_images/test1_images/ds001919/sub-nwu05_T1w.nii.gz
[ 30.  -4. -34.]
[INFO] Given volume is faced with confidence of: 0.8858739733695984
/work/01329/poldrack/data/mriqc-net/data/test_images/test1_images/ds001919/sub-cmrra01_T1w.nii.gz
[ 30.  -4. -34.]
[INFO] Given volume is faced with confidence of: 0.99085533618927
/work/01329/poldrack/data/mriqc-net/data/test_images/test1_images/ds001919/sub-cardiff06_T1w.nii.gz
[ 30.  -4. -34.]
[INFO] Given volume is faced with confidence of: 0.9940774440765381
/work/01329/poldrack/data/mriqc-net/data/test_images/test1_images/ds001919/sub-queensland02_T1w.nii.gz
[ 30.  -4. -34.]
[INFO] Given volume is faced with confidence of: 0.95191788

...

[ 30.  -4. -34.]
[INFO] Given volume is defaced with confidence of: 0.9878728212788701
/work/01329/poldrack/data/mriqc-net/data/test_images/test1_images/ds001919/sub-oxfordFmrib06_T1w.nii.gz
[ 30.  -4. -34.]
[INFO] Given volume is defaced with confidence of: 0.8840519189834595
/work/01329/poldrack/data/mriqc-net/data/test_images/test1_images/ds001919/sub-nwu05_T1w.nii.gz
[ 30.  -4. -34.]
[INFO] Given volume is defaced with confidence of: 0.9995227726467419
/work/01329/poldrack/data/mriqc-net/data/test_images/test1_images/ds001919/sub-cmrra01_T1w.nii.gz
[ 30.  -4. -34.]
[INFO] Given volume is defaced with confidence of: 0.9790994860231876
/work/01329/poldrack/data/mriqc-net/data/test_images/test1_images/ds001919/sub-cardiff06_T1w.nii.gz
[ 30.  -4. -34.]
[INFO] Given volume is defaced with confidence of: 0.6892105340957642
/work/01329/poldrack/data/mriqc-net/data/test_images/test1_images/ds001919/sub-queensland02_T1w.nii.gz
[ 30.  -4. -34.]
[INFO] Given volume is defaced with confidence 

[  32. -130. -130.]
[INFO] Given volume is defaced with confidence of: 0.9991277469671331
/work/01329/poldrack/data/mriqc-net/data/test_images/test1_images/ds001919/sub-brnoPrisma03_T1w.nii.gz
[ 30.  -4. -34.]
[INFO] Given volume is faced with confidence of: 0.5210301876068115
/work/01329/poldrack/data/mriqc-net/data/test_images/test1_images/ds001919/sub-vuiisIngenia01_T1w.nii.gz
[ 30. -34. -34.]
[INFO] Given volume is defaced with confidence of: 0.9999870426845519
/work/01329/poldrack/data/mriqc-net/data/test_images/test1_images/ds001919/sub-oxfordFmrib04_T1w.nii.gz
[ 30.  -4. -34.]
[INFO] Given volume is defaced with confidence of: 0.8790303468704224
/work/01329/poldrack/data/mriqc-net/data/test_images/test1_images/ds001919/sub-amu05_T1w.nii.gz
[ 30.  -4. -34.]
[INFO] Given volume is defaced with confidence of: 0.9862570539116859
/work/01329/poldrack/data/mriqc-net/data/test_images/test1_images/ds001919/sub-ubc05_T1w.nii.gz
[ 30. -34. -34.]
[INFO] Given volume is defaced with confide

[ 30. -34. -34.]
[INFO] Given volume is defaced with confidence of: 0.9999510922571062
/work/01329/poldrack/data/mriqc-net/data/test_images/test1_images/ds001919/sub-brnoPrisma01_T1w.nii.gz
[ 30.  -4. -34.]
[INFO] Given volume is faced with confidence of: 0.9795098304748535
/work/01329/poldrack/data/mriqc-net/data/test_images/test1_images/ds001919/sub-pavia01_T1w.nii.gz
[ 30.  -4. -34.]
[INFO] Given volume is defaced with confidence of: 0.9995172489725519
/work/01329/poldrack/data/mriqc-net/data/test_images/test1_images/ds001919/sub-tokyoSkyra05_T1w.nii.gz
[ 30.  -4. -34.]
[INFO] Given volume is defaced with confidence of: 0.9999964889927924
/work/01329/poldrack/data/mriqc-net/data/test_images/test1_images/ds001919/sub-mpicbs02_T1w.nii.gz
[ 30.  -4. -34.]
[INFO] Given volume is defaced with confidence of: 0.9993099425337277
/work/01329/poldrack/data/mriqc-net/data/test_images/test1_images/ds001919/sub-milan03_T1w.nii.gz
[ 30.  -4. -34.]
[INFO] Given volume is faced with confidence of: 

[ 30.  -4. -34.]
[INFO] Given volume is defaced with confidence of: 0.7392078042030334
/work/01329/poldrack/data/mriqc-net/data/test_images/test1_images/ds001919/sub-juntendo750w03_T1w.nii.gz
[ 32. -34. -34.]
[INFO] Given volume is defaced with confidence of: 0.7820480167865753
/work/01329/poldrack/data/mriqc-net/data/test_images/test1_images/ds001919/sub-sherbrooke02_T1w.nii.gz
[ 30. -34. -34.]
[INFO] Given volume is defaced with confidence of: 0.9190165996551514
/work/01329/poldrack/data/mriqc-net/data/test_images/test1_images/ds001919/sub-sapienza04_T1w.nii.gz
[ 30. -34. -34.]
[INFO] Given volume is defaced with confidence of: 0.9488997161388397
/work/01329/poldrack/data/mriqc-net/data/test_images/test1_images/ds001919/sub-tokyoIngenia07_T1w.nii.gz
[ 30. -34. -34.]
[INFO] Given volume is faced with confidence of: 0.6214795112609863
/work/01329/poldrack/data/mriqc-net/data/test_images/test1_images/ds001919/sub-tokyoSkyra04_T1w.nii.gz
[ 30.  -4. -34.]
[INFO] Given volume is defaced wi

[ 30. -34. -34.]
[INFO] Given volume is defaced with confidence of: 0.9851204715669155
/work/01329/poldrack/data/mriqc-net/data/test_images/test1_images/ds001919/sub-oxfordFmrib09_T1w.nii.gz
[ 30.  -4. -34.]
[INFO] Given volume is defaced with confidence of: 0.9364134520292282
/work/01329/poldrack/data/mriqc-net/data/test_images/test1_images/ds001919/sub-tokyoIngenia05_T1w.nii.gz
[ 30. -34. -34.]
[INFO] Given volume is defaced with confidence of: 0.9999996263163666
/work/01329/poldrack/data/mriqc-net/data/test_images/test1_images/ds001919/sub-cmrra02_T1w.nii.gz
[ 30.  -4. -34.]
[INFO] Given volume is faced with confidence of: 0.7812325358390808
/work/01329/poldrack/data/mriqc-net/data/test_images/test1_images/ds001919/sub-barcelona05_T1w.nii.gz
[ 30.  -4. -34.]
[INFO] Given volume is defaced with confidence of: 0.9942422769963741
/work/01329/poldrack/data/mriqc-net/data/test_images/test1_images/ds001919/sub-sapienza01_T1w.nii.gz
[ 30. -34. -34.]
[INFO] Given volume is faced with confid

In [10]:
import pandas as pd

# inference_results_path = 'test_inference_face_refaced.csv'
# data = pd.read_csv(inference_results_path)
# data.rename(columns={'Unnamed: 0':'volume'}, inplace=True 

# true_labels = pd.read_csv('/work/06850/sbansal6/maverick2/mriqc-net/2.5D/csv/faced/all.csv')
#             volumes, tlabels = true_labels['X'].values, true_labels['Y'].values

# tvolumes = {}

# for v in range(len(volumes)):
    
#     tvolumes[volumes[v].split('/')[-2] + '/' + volumes[v].split('/')[-1]] = tlabels[v]
    
true_labels = pd.read_csv('test1_images_gt.csv')
tl_dict = {}
for i, row in enumerate(true_labels.values):
    
    dataset, volume, label = row
    l = 0 if 'deface' in label else 1
    tl_dict[str(dataset) + '/' + str(volume)] = l
    
# print(tl_dict)
percentage = [25, 50, 75, 100]
for per in percentage:
    print("Percentage Data: ", per)
    predicted = pd.read_csv('test_inference_dstest_model_fdefaced_{}.csv'.format(per))
    predicted.rename(columns={'Unnamed: 0':'volume'}, inplace=True )

    # remove_vols = [
    #     'ds001919',
    #     'ds000148'
    # ]
    correct = 0
    incorrect = {}
    for i, row in enumerate(predicted.values):
        volume, confidence, label = row

        if volume in tl_dict:
            if label == tl_dict[volume]:
                correct += 1
            else:
    #             if 'ds001919' not in volume and 'ds000206' not in volume:
                if per == 100:
                    print(volume, confidence, label, tl_dict[volume])
                incorrect[volume] = {}
                incorrect[volume]['label'] = label
                incorrect[volume]['confidence'] = confidence

    print(len(incorrect))

Percentage Data:  25
110
Percentage Data:  50
87
Percentage Data:  75
115
Percentage Data:  100
ds002330/sub-10_T1w.nii.gz 0.7335546016693115 0.0 1
ds002330/sub-65_T1w.nii.gz 0.9055495411157608 0.0 1
ds002330/sub-47_T1w.nii.gz 0.609892725944519 1.0 0
ds002330/sub-52_T1w.nii.gz 0.9551036283373832 0.0 1
ds002330/sub-45_T1w.nii.gz 0.9600865580141544 0.0 1
ds002330/sub-04_T1w.nii.gz 0.67268967628479 0.0 1
ds002330/sub-44_T1w.nii.gz 0.7937524914741516 0.0 1
ds002330/sub-24_T1w.nii.gz 0.5360963642597198 0.0 1
ds002330/sub-51_T1w.nii.gz 0.7607027292251587 0.0 1
ds002330/sub-06_T1w.nii.gz 0.95715818926692 0.0 1
ds002330/sub-31_T1w.nii.gz 0.9943286497145892 0.0 1
ds002330/sub-02_T1w.nii.gz 0.6914727389812471 0.0 1
ds002330/sub-21_T1w.nii.gz 0.9653783291578292 0.0 1
ds002330/sub-61_T1w.nii.gz 0.7501114159822464 0.0 1
ds002330/sub-30_T1w.nii.gz 0.5178702473640442 0.0 1
ds002330/sub-59_T1w.nii.gz 0.9668572954833508 0.0 1
ds002330/sub-14_T1w.nii.gz 0.8073982298374176 0.0 1
ds002330/sub-09_T1w.nii.g

In [10]:
print(correct, len(incorrect))

265 49


In [ ]:
cpaths = glob.glob('/work/06850/sbansal6/maverick2/mriqc-shared/test_conformed/*/*')

labels = []
for c in cpaths:
    name = '/'.join(c.rsplit('/', 2)[-2:])
    print(name, tl_dict[name], incorrect[name]['label'])
    labels.append(tl_dict[name])
    
print(len(labels))

In [ ]:
import sys, os
sys.path.append('../defacing')

import numpy as np
# from scipy.misc import toimage
from PIL import Image, ImageFont, ImageDraw
from helpers.utils import save_vol, load_vol
from preprocessing.conform import conform_data
from preprocessing.normalization import clip, standardize, normalize
import glob
import matplotlib.pyplot as plt

conform_size = (128, 128, 128)
conform_zoom = (1.5, 1.5, 1.5)

testpath = '/work/01329/poldrack/data/mriqc-net/data/test_images/test1_images'

# for f in incorrect:
    
#     dname = f.split('/')[0]
#     fname = f.split('/')[1]
#     print(dname, fname)
    
#     file = os.path.join(testpath, f)
#     print(file)
    
#     cpath = '/work/06850/sbansal6/maverick2/mriqc-shared/test_conformed'
#     outfile = os.path.join(cpath, dname)
#     print(outfile)
#     os.makedirs(outfile, exist_ok=True)
    
#     outfile = os.path.join(outfile, fname)
#     print(outfile)
#     conform_data(file, 
#                  out_file=outfile, 
#                  out_size=conform_size, 
#                  out_zooms=conform_zoom)
    
#     vol,_,_ = load_vol(outfile)
    
#     vol = np.array(vol)
#     vol = np.transpose(vol, axes=[1, 2, 0])
    
#     plt.figure(figsize=(20, 10))
#     for i in range(1, 11):
#         plt.subplot(1, 10, i)
#         plt.imshow(vol[i,:,:])
    
#     plt.show()
    
cpaths = glob.glob('/work/06850/sbansal6/maverick2/mriqc-shared/test_conformed/*/*')

images = []
nslices = 16

for v in cpaths:
    
    
    vol, _, _ = load_vol(v)
    vol = np.array(vol)
    vol = clip(vol, q=90)
    vol = normalize(vol)
    vol = standardize(vol)
    
    # get Coronal
    vol = np.transpose(vol, (2, 0, 1))
#     plt.figure(figsize=(20, 10))
#     for i in range(1, 31):
#         plt.subplot(1, 31, i)
#         plt.imshow(vol[i+24,:,:])
#     plt.show()


    for i in range(1, nslices+1):
        im = vol[i+64, :, :]
        images.append(im)
        
print(len(images))

images = np.array(images)*255

total_width = 128*nslices
total_height = 128*len(cpaths)
    
new_im = Image.new('RGB',(total_height, total_width))

nrows = len(cpaths)
ncols = nslices
im_size = 128

print(new_im)
for im in range(len(images)):
    imag = Image.fromarray(images[im])
#     print(images[im].shape)
    x, y = im_size*int(im/ncols), im_size*int(im%ncols)
    new_im.paste(imag, (x,y))
    if y % 128 == 0:
        draw = ImageDraw.Draw(new_im)
        text = str(labels[int(im % len(images)/nslices)])
#         print(text)
        draw.text((x,y), text)
    

new_im.save('mosaic/' + 'test_images_misclassified_axial.png')
plt.figure(figsize=(20, 10))
plt.imshow(new_im)
plt.show()

In [ ]:
import os 

f = open("discovered.txt", "r")
discovered = f.read().splitlines()

defaced = os.listdir('/work/01329/poldrack/data/mriqc-net/data/defaced')

for ds in defaced:
    for dis in discovered:
        if dis in ds:
            print(dis)

In [ ]:
correct = 0
incorrect = 0

for vol in data['volume'].values:
    conf = data.loc[data['volume'] == vol, 'confidence'].item()
    label = data.loc[data['volume'] == vol, 'label'].item()
    
    if vol in tvolumes:
        if label == tvolumes[vol]:
            correct += 1
        else:
            incorrect += 1
    else:
        print(vol)
    

In [4]:
!pip install --user --upgrade tensorboard

     |████████████████████████████████| 6.8 MB 4.2 MB/s eta 0:00:01
     |████████████████████████████████| 779 kB 38.6 MB/s eta 0:00:01
ERROR: tensorflow-gpu 2.1.0 has requirement tensorboard<2.2.0,>=2.1.0, but you'll have tensorboard 2.3.0 which is incompatible.
ERROR: tensorflow 2.1.0 has requirement tensorboard<2.2.0,>=2.1.0, but you'll have tensorboard 2.3.0 which is incompatible.
  Attempting uninstall: tensorboard
    Found existing installation: tensorboard 2.1.0
    Uninstalling tensorboard-2.1.0:
      Successfully uninstalled tensorboard-2.1.0
You should consider upgrading via the '/work/06850/sbansal6/anaconda3/bin/python -m pip install --upgrade pip' command.


In [5]:
from packaging import version

import pandas as pd
from matplotlib import pyplot as plt
import seaborn as sns
from scipy import stats
import tensorboard as tb

# tensorboard at https://tensorboard.dev/experiment/wJCGhllKTG2Mjr3w811Xnw/

experiment_id = "wJCGhllKTG2Mjr3w811Xnw"
experiment = tb.data.experimental.ExperimentFromDev(experiment_id)
df = experiment.get_scalars()
df

,run,tag,step,value
0,train_test_fold_1/tb_logs/axial/train,epoch_accuracy,0,0.958841
1,train_test_fold_1/tb_logs/axial/train,epoch_accuracy,1,0.993394
2,train_test_fold_1/tb_logs/axial/train,epoch_accuracy,2,0.991870
3,train_test_fold_1/tb_logs/axial/train,epoch_accuracy,3,0.993902
4,train_test_fold_1/tb_logs/axial/train,epoch_accuracy,4,0.993902
5,train_test_fold_1/tb_logs/axial/train,epoch_accuracy,5,0.997459
6,train_test_fold_1/tb_logs/axial/train,epoch_accuracy,6,0.996443
7,train_test_fold_1/tb_logs/axial/train,epoch_accuracy,7,0.990346
8,train_test_fold_1/tb_logs/axial/train,epoch_accuracy,8,0.994919
9,train_test_fold_1/tb_logs/axial/train,epoch_accuracy,9,0.995427


In [17]:
temp = df[df["run"].str.contains('combined/validation')]
# print(temp[temp["tag"].str.contains('epoch_accuracy')])

print(temp[(temp['tag'].str.contains('epoch_accuracy')) & (temp['step'] == 14)])

                                                  run             tag  step  \
419     train_test_fold_1/tb_logs/combined/validation  epoch_accuracy    14   
1499   train_test_fold_10/tb_logs/combined/validation  epoch_accuracy    14   
2579   train_test_fold_11/tb_logs/combined/validation  epoch_accuracy    14   
3659   train_test_fold_12/tb_logs/combined/validation  epoch_accuracy    14   
4739   train_test_fold_13/tb_logs/combined/validation  epoch_accuracy    14   
5819   train_test_fold_14/tb_logs/combined/validation  epoch_accuracy    14   
6899   train_test_fold_15/tb_logs/combined/validation  epoch_accuracy    14   
7979    train_test_fold_2/tb_logs/combined/validation  epoch_accuracy    14   
9059    train_test_fold_3/tb_logs/combined/validation  epoch_accuracy    14   
10139   train_test_fold_4/tb_logs/combined/validation  epoch_accuracy    14   
11219   train_test_fold_5/tb_logs/combined/validation  epoch_accuracy    14   
12299   train_test_fold_6/tb_logs/combined/validatio

In [ ]:
csv_paths = glob.glob('/work/06850/sbansal6/maverick2/mriqc-shared/experiment_data_all/F15_csv/train/*')

# combined_train_csv = pd.read_csv()
average_train_accuracy = 0
for csv in csv_paths:
    accuracies = pd.read_csv(csv)["Value"].values
    average_train_accuracy += accuracies[-1]
    
average_train_accuracy = average_train_accuracy/15
print("Average 15 fold training accuracy for combined model: ", average_train_accuracy)

csv_paths = glob.glob('/work/06850/sbansal6/maverick2/mriqc-shared/experiment_data_all/F15_csv/validation/*')

# combined_train_csv = pd.read_csv()
average_valid_accuracy = 0
for csv in csv_paths:
    accuracies = pd.read_csv(csv)["Value"].values
    average_valid_accuracy += accuracies[-1]
#     print(accuracies[-1])
    
average_valid_accuracy = average_valid_accuracy/15
print("Average 15 fold validation accuracy for combined model: ", average_valid_accuracy)